In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import validation_curve
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import ExtraTreesClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import roc_curve,auc,roc_auc_score
import xgboost as xgb
from xgboost import XGBClassifier
from vecstack import stacking
import numpy as np

In [2]:
train_x= pd.read_csv("train_test_boxcox_trans11_mean35_importance44_order6_comd4.csv",encoding="big5")

In [3]:
train_y=pd.read_csv("train_y.csv")

In [34]:
X_train, X_test, y_train, y_test = train_test_split(
    train_x[:100000], train_y, test_size=0.6, shuffle=False)

In [17]:
train_y["Y1"]=train_y["Y1"].map({"N":0,"Y":1})

In [35]:
clf = LogisticRegression(random_state=0, solver='lbfgs',
                         multi_class='multinomial').fit(X_train, y_train)

C:\Users\a2279\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [36]:
logistic_regr = LogisticRegression()
logistic_regr.fit(X_train, y_train)

C:\Users\a2279\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\a2279\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [39]:
log_predictions = logistic_regr.predict(X_test)
y_predprob= logistic_regr.predict_proba(X_test)
accuracy = logistic_regr.score(X_train, y_train)
print(accuracy)

0.979875


In [43]:
y_predprob

array([[0.97175941, 0.02824059],
       [0.98304636, 0.01695364],
       [0.98304636, 0.01695364],
       ...,
       [0.97705309, 0.02294691],
       [0.9796552 , 0.0203448 ],
       [0.5       , 0.5       ]])

In [45]:
roc_auc_score(y_test, y_predprob[:,0])

0.535723572590638

In [44]:
X_train.shape, X_test.shape, len(y_train), len(y_test)

((40000, 171), (60000, 171), 40000, 60000)

In [16]:
forest=RandomForestClassifier(criterion='entropy',n_estimators=1000,random_state=1,n_jobs=2)
skf = StratifiedShuffleSplit(n_splits=5)

In [17]:
for train_index, test_index in skf.split(X_train_1,y_train):
    X_train1, X_test1 = X_train_1.loc[train_index], X_train_1.loc[test_index]
    y_train1, y_test1 = y_train.loc[train_index], y_train.loc[test_index]
    forest.fit(X_train1,y_train1)
    y_predprob1 =forest.predict_proba(X_test1)[:,1]
    print('Auc:%.3f' % (roc_auc_score(y_test1, y_predprob1)))

Auc:0.840
Auc:0.843
Auc:0.854
Auc:0.838
Auc:0.846


CB

In [30]:
def CB_200(X_train, y_train): 
    print()   
    
    import warnings
    warnings.filterwarnings("ignore")
    
    from scipy.stats import uniform as sp_randFloat
    from scipy.stats import randint as sp_randInt

    X_train1, X_test1, y_train1, y_test1 = train_test_split(X_train, y_train, test_size=0.25)
    
    model = CatBoostClassifier(random_state=27)
    parameters = {'depth'         : sp_randInt(4, 10),
                  'learning_rate' : sp_randFloat(),
                  'iterations'    : sp_randInt(10, 1000),
                  'l2_leaf_reg':[3,1,5,10,100],
                 }
    
    randm = RandomizedSearchCV(estimator=model, param_distributions=parameters, 
                               cv=5, n_iter=10, n_jobs=-1)
    randm.fit(X_train1, y_train1)

    # Results from Random Search
    print("\n========================================================")
    print(" Results from Random Search " )
    print("========================================================")    
    
    print("\n The best estimator across ALL searched params:\n",
          randm.best_estimator_)
    
    print("\n The best score across ALL searched params:\n",
          randm.best_score_)
    
    print("\n The best parameters across ALL searched params:\n",
          randm.best_params_)
    
    print("\n ========================================================")
CB_200(X_train_1, y_train)


0:	learn: 0.1497445	total: 112ms	remaining: 1m 9s
1:	learn: 0.0963435	total: 177ms	remaining: 54.8s
2:	learn: 0.0851116	total: 241ms	remaining: 49.8s
3:	learn: 0.0812808	total: 311ms	remaining: 48.1s
4:	learn: 0.0795284	total: 383ms	remaining: 47.3s
5:	learn: 0.0778527	total: 447ms	remaining: 45.9s
6:	learn: 0.0769140	total: 521ms	remaining: 45.8s
7:	learn: 0.0759848	total: 583ms	remaining: 44.8s
8:	learn: 0.0756633	total: 645ms	remaining: 43.9s
9:	learn: 0.0749140	total: 708ms	remaining: 43.4s
10:	learn: 0.0746513	total: 773ms	remaining: 42.9s
11:	learn: 0.0740468	total: 837ms	remaining: 42.6s
12:	learn: 0.0734147	total: 902ms	remaining: 42.3s
13:	learn: 0.0723716	total: 973ms	remaining: 42.3s
14:	learn: 0.0713820	total: 1.04s	remaining: 41.9s
15:	learn: 0.0708578	total: 1.1s	remaining: 41.6s
16:	learn: 0.0703635	total: 1.16s	remaining: 41.4s
17:	learn: 0.0699122	total: 1.23s	remaining: 41.1s
18:	learn: 0.0694242	total: 1.29s	remaining: 41s
19:	learn: 0.0689507	total: 1.36s	remaining

163:	learn: 0.0190700	total: 10.8s	remaining: 30.3s
164:	learn: 0.0188891	total: 10.9s	remaining: 30.2s
165:	learn: 0.0186406	total: 11s	remaining: 30.2s
166:	learn: 0.0184442	total: 11s	remaining: 30.1s
167:	learn: 0.0183240	total: 11.1s	remaining: 30s
168:	learn: 0.0181197	total: 11.2s	remaining: 30s
169:	learn: 0.0179329	total: 11.2s	remaining: 29.9s
170:	learn: 0.0178387	total: 11.3s	remaining: 29.8s
171:	learn: 0.0177231	total: 11.4s	remaining: 29.7s
172:	learn: 0.0176132	total: 11.4s	remaining: 29.7s
173:	learn: 0.0174095	total: 11.5s	remaining: 29.6s
174:	learn: 0.0171670	total: 11.6s	remaining: 29.5s
175:	learn: 0.0170356	total: 11.6s	remaining: 29.5s
176:	learn: 0.0169493	total: 11.7s	remaining: 29.4s
177:	learn: 0.0167792	total: 11.8s	remaining: 29.3s
178:	learn: 0.0166055	total: 11.8s	remaining: 29.3s
179:	learn: 0.0162336	total: 11.9s	remaining: 29.2s
180:	learn: 0.0160481	total: 12s	remaining: 29.2s
181:	learn: 0.0159835	total: 12s	remaining: 29.1s
182:	learn: 0.0158449	to

324:	learn: 0.0039817	total: 21.6s	remaining: 19.7s
325:	learn: 0.0039484	total: 21.7s	remaining: 19.7s
326:	learn: 0.0039186	total: 21.7s	remaining: 19.6s
327:	learn: 0.0038965	total: 21.8s	remaining: 19.6s
328:	learn: 0.0038461	total: 21.9s	remaining: 19.5s
329:	learn: 0.0038212	total: 22s	remaining: 19.4s
330:	learn: 0.0037830	total: 22s	remaining: 19.4s
331:	learn: 0.0037577	total: 22.1s	remaining: 19.3s
332:	learn: 0.0037346	total: 22.1s	remaining: 19.2s
333:	learn: 0.0037252	total: 22.2s	remaining: 19.2s
334:	learn: 0.0037052	total: 22.3s	remaining: 19.1s
335:	learn: 0.0036835	total: 22.3s	remaining: 19s
336:	learn: 0.0036368	total: 22.4s	remaining: 18.9s
337:	learn: 0.0036004	total: 22.5s	remaining: 18.9s
338:	learn: 0.0035833	total: 22.5s	remaining: 18.8s
339:	learn: 0.0035091	total: 22.6s	remaining: 18.7s
340:	learn: 0.0034729	total: 22.7s	remaining: 18.7s
341:	learn: 0.0034506	total: 22.7s	remaining: 18.6s
342:	learn: 0.0034280	total: 22.8s	remaining: 18.6s
343:	learn: 0.0033

483:	learn: 0.0014392	total: 32.1s	remaining: 9.16s
484:	learn: 0.0014323	total: 32.2s	remaining: 9.09s
485:	learn: 0.0014287	total: 32.3s	remaining: 9.03s
486:	learn: 0.0014263	total: 32.3s	remaining: 8.96s
487:	learn: 0.0014128	total: 32.4s	remaining: 8.89s
488:	learn: 0.0014087	total: 32.5s	remaining: 8.83s
489:	learn: 0.0014041	total: 32.5s	remaining: 8.76s
490:	learn: 0.0013958	total: 32.6s	remaining: 8.69s
491:	learn: 0.0013850	total: 32.7s	remaining: 8.63s
492:	learn: 0.0013743	total: 32.7s	remaining: 8.56s
493:	learn: 0.0013692	total: 32.8s	remaining: 8.49s
494:	learn: 0.0013659	total: 32.8s	remaining: 8.43s
495:	learn: 0.0013597	total: 32.9s	remaining: 8.36s
496:	learn: 0.0013570	total: 33s	remaining: 8.29s
497:	learn: 0.0013545	total: 33s	remaining: 8.23s
498:	learn: 0.0013484	total: 33.1s	remaining: 8.16s
499:	learn: 0.0013438	total: 33.2s	remaining: 8.09s
500:	learn: 0.0013376	total: 33.2s	remaining: 8.03s
501:	learn: 0.0013355	total: 33.3s	remaining: 7.96s
502:	learn: 0.00

SVM

In [ ]:
model = SVC()
model.fit(X_train, y_train)

In [ ]:
# defining parameter range 
param_grid = {'C': [0.1, 1, 10, 100, 1000],  
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
              'kernel': ['rbf']}  
  
grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3) 
  
# fitting the model for grid search 
grid.fit(X_train, y_train)

RF

In [20]:
rfc = RandomForestClassifier()
rfc.fit(X_train[predictors],y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [27]:
# number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 1000, num = 31)]
# number of features at every split
max_features = ['auto']

# max depth
max_depth = [int(x) for x in np.linspace(100, 500, num = 11)]
max_depth.append(None)
# create random grid
random_grid = {
 'n_estimators': n_estimators,
 'max_features': max_features,
 'max_depth': max_depth
 }
# Random search of parameters
rfc_random = RandomizedSearchCV(estimator = rfc, param_distributions = random_grid, n_iter = 100, 
                                scoring='roc_auc', cv = 5, verbose=2, random_state=0, n_jobs = -1)
# Fit the model
rfc_random.fit(X_train[predictors], y_train)
# print results
print(rfc_random.best_params_)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed: 12.7min
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed: 55.9min
[Parallel(n_jobs=-1)]: Done 341 tasks      | elapsed: 129.3min
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed: 187.2min finished


{'n_estimators': 1000, 'max_features': 'auto', 'max_depth': 180}


In [29]:
# X_train_1 = X_train[predictors]
# X_test_1 = X_test[predictors]
# for train_index, test_index in skf.split(X_train_1,y_train):
#     X_train1, X_test1 = X_train_1.loc[train_index], X_train_1.loc[test_index]
#     y_train1, y_test1 = y_train.loc[train_index], y_train.loc[test_index]
#     forest.fit(X_train1,y_train1)
#     y_predprob1 =forest.predict_proba(X_test1)[:,1]
#     print('Auc:%.3f' % (roc_auc_score(y_test1, y_predprob1)))

Auc:0.827
Auc:0.831
Auc:0.826
Auc:0.817
Auc:0.808


In [28]:
X_train_1 = X_train[predictors]
for train_index, test_index in skf.split(X_train_1,y_train):
    X_train1, X_test1 = X_train_1.loc[train_index], X_train_1.loc[test_index]
    y_train1, y_test1 = y_train.loc[train_index], y_train.loc[test_index]
    forest.fit(X_train1,y_train1)
    y_predprob1 =forest.predict_proba(X_test1)[:,1]
    print('Auc:%.3f' % (roc_auc_score(y_test1, y_predprob1)))

Auc:0.820
Auc:0.837
Auc:0.836
Auc:0.807
Auc:0.841


ET

In [28]:
modelET = ExtraTreesClassifier(random_state=27, 
                               n_estimators=500, 
                               max_depth=3)
params_opt = {'n_estimators':range(100, 500, 80), 
              'max_depth':[1,2,3,4,5]
             }
gridSearchCV = GridSearchCV(estimator = modelET, param_grid = params_opt, scoring='roc_auc', 
                            n_jobs=-1, iid=False, verbose=1, cv=5)
gridSearchCV.fit(X_train_1,y_train)
gridSearchCV.cv_results_, gridSearchCV.best_params_, gridSearchCV.best_score_

Fitting 5 folds for each of 25 candidates, totalling 125 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   31.5s
[Parallel(n_jobs=-1)]: Done 125 out of 125 | elapsed:  4.8min finished


({'mean_fit_time': array([ 5.3385416 ,  9.08357439, 11.76050596, 15.51620541, 17.89692678,
          8.41351266, 13.7950213 , 19.2763103 , 24.59130759, 29.94116211,
         11.42300487, 19.21360335, 26.68606849, 34.95318174, 41.95680771,
         14.0462903 , 24.59618797, 34.55601072, 44.96857352, 53.99159951,
         17.28133116, 30.15290394, 42.16688709, 50.44844975, 43.29947052]),
  'std_fit_time': array([0.33219076, 0.84468309, 0.59660435, 0.67425307, 0.82091232,
         0.33255458, 0.60478159, 0.65410062, 0.70174042, 0.4621104 ,
         0.40189283, 0.75462946, 1.04590209, 1.06408972, 1.45935693,
         0.58268068, 0.92561401, 1.8094487 , 1.75769248, 1.67088119,
         0.65058195, 1.33579019, 1.73508328, 1.56918916, 7.55769461]),
  'mean_score_time': array([0.38078179, 0.46298294, 0.58546395, 0.72778454, 0.83498206,
         0.32520366, 0.47655954, 0.62847905, 0.7854115 , 0.90405293,
         0.3389627 , 0.49393945, 0.65525866, 0.82278805, 0.96369905,
         0.3506937 , 0

In [29]:
modelET = ExtraTreesClassifier(random_state=27, 
                               n_estimators=180, 
                               max_depth=3)
params_opt = {'max_depth':[5,6,7]}
gridSearchCV = GridSearchCV(estimator = modelET, param_grid = params_opt, scoring='roc_auc', 
                            n_jobs=-1, iid=False, verbose=1, cv=5)
gridSearchCV.fit(X_train_1,y_train)
gridSearchCV.cv_results_, gridSearchCV.best_params_, gridSearchCV.best_score_

Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 out of  15 | elapsed:   34.9s remaining:   30.5s
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:   47.3s finished


({'mean_fit_time': array([28.67140088, 34.4317955 , 26.12527914]),
  'std_fit_time': array([1.35394963, 0.94595168, 9.51500409]),
  'mean_score_time': array([0.55469046, 0.47084966, 0.33392816]),
  'std_score_time': array([0.05868885, 0.10959621, 0.06933249]),
  'param_max_depth': masked_array(data=[5, 6, 7],
               mask=[False, False, False],
         fill_value='?',
              dtype=object),
  'params': [{'max_depth': 5}, {'max_depth': 6}, {'max_depth': 7}],
  'split0_test_score': array([0.71147449, 0.70287474, 0.70583648]),
  'split1_test_score': array([0.96085517, 0.96112085, 0.95903801]),
  'split2_test_score': array([0.80946543, 0.794375  , 0.78550893]),
  'split3_test_score': array([0.80006237, 0.80115619, 0.80217946]),
  'split4_test_score': array([0.77522009, 0.78007634, 0.78255051]),
  'mean_test_score': array([0.81141551, 0.80792063, 0.80702268]),
  'std_test_score': array([0.08217448, 0.08427276, 0.08298493]),
  'rank_test_score': array([1, 2, 3])},
 {'max_depth'

LGBM

In [23]:
model = LGBMClassifier(
    boosting_type="gbdt",
    learning_rate=0.1,
    is_unbalance=True, 
    random_state=27, 
    n_estimators=50,
    num_leaves=30, 
    max_depth=8,
    feature_fraction=0.5,  
    bagging_fraction=0.8, 
    bagging_freq=15   
)

In [25]:
params_opt = {'n_estimators':range(100, 500, 80), 
              'num_leaves':range(5,30,5), 
              'learning_rate':[0.001,0.01,0.1]
             }
gridSearchCV = GridSearchCV(estimator = model, param_grid = params_opt, scoring='roc_auc', 
                            n_jobs=-1, iid=False, verbose=1, cv=5)
gridSearchCV.fit(X_train_1,y_train)
gridSearchCV.cv_results_, gridSearchCV.best_params_, gridSearchCV.best_score_

Fitting 5 folds for each of 75 candidates, totalling 375 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
C:\Users\Brian\Anaconda3\lib\site-packages\joblib\externals\loky\process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   33.6s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:  4.7min
[Parallel(n_jobs=-1)]: Done 375 out of 375 | elapsed: 10.3min finished


({'mean_fit_time': array([ 9.01801519, 11.75746388, 11.0667695 , 11.14792943, 11.44845018,
         12.24379053, 14.36819553, 15.49451399, 16.50562105, 16.85259867,
         15.75742955, 18.77928629, 20.48294911, 21.60422025, 22.58053455,
         19.51264739, 23.45671401, 25.67719855, 27.20014114, 28.6295362 ,
         22.90645576, 27.95841131, 30.65908232, 32.7245358 , 34.72006531,
          7.82628088,  9.33955469, 10.2946826 , 10.53322101, 10.86843605,
         11.55766397, 13.73113036, 14.77416115, 15.6705091 , 16.08237958,
         14.89478126, 17.93484235, 19.41834025, 20.49932718, 21.23013253,
         18.08413692, 21.98754282, 23.87640247, 25.40033031, 26.30841141,
         21.6320333 , 26.25731759, 28.68722978, 30.43424244, 32.37018075,
          7.32607732,  9.33995757, 10.15225444, 11.13338633, 11.08253965,
         11.17356629, 12.04258065, 12.71096249, 13.39356942, 13.99567513,
         13.27046137, 14.81768103, 16.29653301, 17.55660148, 18.50011029,
         16.23015909,

In [27]:
model = LGBMClassifier(
    boosting_type="gbdt",
    learning_rate=0.01,
    is_unbalance=True, 
    random_state=27, 
    n_estimators=420,
    num_leaves=30, 
    max_depth=8,
    feature_fraction=0.5,  
    bagging_fraction=0.8, 
    bagging_freq=15   
)
params_opt = {'num_leaves':range(3,8,1)}
gridSearchCV = GridSearchCV(estimator = model, param_grid = params_opt, scoring='roc_auc', 
                            n_jobs=-1, iid=False, verbose=1, cv=5)
gridSearchCV.fit(X_train_1,y_train)
gridSearchCV.cv_results_, gridSearchCV.best_params_, gridSearchCV.best_score_

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:   54.9s finished


({'mean_fit_time': array([17.34885798, 19.43082209, 21.32178116, 21.30072827, 19.96880484]),
  'std_fit_time': array([0.4733863 , 0.28025394, 0.53236622, 0.39591078, 2.64321331]),
  'mean_score_time': array([0.75380201, 0.89963202, 0.95820656, 0.84385991, 0.70545664]),
  'std_score_time': array([0.01712541, 0.03436634, 0.02621853, 0.10757379, 0.05263636]),
  'param_num_leaves': masked_array(data=[3, 4, 5, 6, 7],
               mask=[False, False, False, False, False],
         fill_value='?',
              dtype=object),
  'params': [{'num_leaves': 3},
   {'num_leaves': 4},
   {'num_leaves': 5},
   {'num_leaves': 6},
   {'num_leaves': 7}],
  'split0_test_score': array([0.72956001, 0.73542392, 0.73564764, 0.73679777, 0.73572742]),
  'split1_test_score': array([0.95546301, 0.95375383, 0.95248922, 0.95039911, 0.94959349]),
  'split2_test_score': array([0.82291511, 0.82325644, 0.81917385, 0.80452902, 0.80077411]),
  'split3_test_score': array([0.80053941, 0.80516901, 0.80671467, 0.80825753

XGB

In [30]:
def modelfit(alg, dtrain, dtest, predictors,useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(dtrain[predictors].values, label=dtest.values)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
            metrics='auc', early_stopping_rounds=early_stopping_rounds)
        alg.set_params(n_estimators=cvresult.shape[0])

#Fit the algorithm on the data
    alg.fit(dtrain[predictors], dtest, eval_metric='auc')

#Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors])
    dtrain_predprob = alg.predict_proba(dtrain[predictors])[:,1]

#Print model report:
    print ("\nModel Report")
    print ('Accuracy : %.4f' % accuracy_score(dtest.values, dtrain_predictions))
    print ("AUC Score (Train): %f" % roc_auc_score(dtest, dtrain_predprob))

In [40]:
#Choose all predictors except target & IDcols
predictors = [x for x in X_train.columns if x not in ['CUS_ID']]
xgb1 = XGBClassifier(
    learning_rate=0.1,
    n_estimators=1000,
    max_depth=5,
    min_child_weight=1,
    gamma=0,
    subsample=0.8,
    colsample_bytree=0.8,
    objective= 'binary:logistic',
    nthread=4,
    scale_pos_weight=1,
    seed=27)
modelfit(xgb1, X_train, y_train, predictors)


Model Report
Accuracy : 0.9810
AUC Score (Train): 0.902024


In [32]:
xgb1.get_booster().best_iteration

90

In [33]:
param_test1 = {
 'max_depth':range(3,10,2),
 'min_child_weight':range(1,7,2)
}
gsearch1 = GridSearchCV(estimator = XGBClassifier(learning_rate =0.1, n_estimators=90, max_depth=5,
                                                  min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
                                                  objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27),
                        param_grid = param_test1, scoring='roc_auc', n_jobs=4, iid=False, cv=5)
gsearch1.fit(X_train[predictors],y_train)
gsearch1.cv_results_, gsearch1.best_params_, gsearch1.best_score_

({'mean_fit_time': array([ 47.45100808,  47.27254133,  47.42732964,  68.2144546 ,
          68.41673079,  66.30219069,  87.62726116,  90.06335034,
          85.87611399, 114.94881701, 110.94276462,  99.77044449]),
  'std_fit_time': array([ 2.08342854,  2.64257266,  3.08778569,  1.92349328,  9.07463639,
          4.53583774, 13.28666232,  6.91490245,  7.4054536 , 10.46633807,
          4.64553437, 10.54492655]),
  'mean_score_time': array([0.37042556, 0.39984765, 0.41760116, 0.39655538, 0.42737842,
         0.40932498, 0.40114388, 0.44163775, 0.40972109, 0.44752054,
         0.45251265, 0.3462677 ]),
  'std_score_time': array([0.02782782, 0.03635807, 0.03147896, 0.0057326 , 0.04172159,
         0.01795886, 0.02876277, 0.0313205 , 0.02515483, 0.01631292,
         0.02064136, 0.07956265]),
  'param_max_depth': masked_array(data=[3, 3, 3, 5, 5, 5, 7, 7, 7, 9, 9, 9],
               mask=[False, False, False, False, False, False, False, False,
                     False, False, False, False]

In [36]:
param_test2 = {
 'max_depth':[1,2,3],
 'min_child_weight':[0,0.5,1]
}
gsearch2 = GridSearchCV(estimator = XGBClassifier(learning_rate=0.1, n_estimators=90, max_depth=5,
                                                  min_child_weight=2, gamma=0, subsample=0.8, colsample_bytree=0.8,
                                                  objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27),
                        param_grid = param_test2, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch2.fit(X_train[predictors],y_train)
gsearch2.cv_results_, gsearch2.best_params_, gsearch2.best_score_

({'mean_fit_time': array([28.55199804, 28.20700002, 29.47006245, 39.58900595, 38.25954199,
         39.69657664, 49.24365983, 49.21841726, 42.96882124]),
  'std_fit_time': array([1.62893727, 0.45131866, 0.97265334, 1.08286455, 1.86923054,
         1.5895827 , 1.26098069, 1.74279572, 8.92573658]),
  'mean_score_time': array([0.34938164, 0.36643853, 0.39486127, 0.40542951, 0.38228927,
         0.41181622, 0.38877568, 0.41271224, 0.31886196]),
  'std_score_time': array([0.01901408, 0.01265771, 0.02473553, 0.04334229, 0.02289458,
         0.03305986, 0.02959501, 0.02458769, 0.07189517]),
  'param_max_depth': masked_array(data=[1, 1, 1, 2, 2, 2, 3, 3, 3],
               mask=[False, False, False, False, False, False, False, False,
                     False],
         fill_value='?',
              dtype=object),
  'param_min_child_weight': masked_array(data=[0, 0.5, 1, 0, 0.5, 1, 0, 0.5, 1],
               mask=[False, False, False, False, False, False, False, False,
                     Fa

In [38]:
param_test3 = {
    'gamma':[i/100.0 for i in range(0,5)]
}
gsearch3 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=90, max_depth=2,
                                                  min_child_weight=0, gamma=0, subsample=0.8, colsample_bytree=0.8,
                                                  objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27),
                        param_grid = param_test3, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch3.fit(X_train[predictors],y_train)
gsearch3.cv_results_, gsearch3.best_params_, gsearch3.best_score_

({'mean_fit_time': array([36.95897546, 40.06915369, 38.23207617, 40.41996136, 35.58239865]),
  'std_fit_time': array([4.00078941, 2.72207851, 2.0577844 , 1.47068363, 6.6925914 ]),
  'mean_score_time': array([0.38299108, 0.38448782, 0.36952605, 0.39396272, 0.32850585]),
  'std_score_time': array([0.02672424, 0.02843261, 0.017506  , 0.0084148 , 0.08666601]),
  'param_gamma': masked_array(data=[0.0, 0.01, 0.02, 0.03, 0.04],
               mask=[False, False, False, False, False],
         fill_value='?',
              dtype=object),
  'params': [{'gamma': 0.0},
   {'gamma': 0.01},
   {'gamma': 0.02},
   {'gamma': 0.03},
   {'gamma': 0.04}],
  'split0_test_score': array([0.74412915, 0.74412915, 0.74412915, 0.74412915, 0.74412915]),
  'split1_test_score': array([0.94954892, 0.94954892, 0.94954892, 0.94954892, 0.94954892]),
  'split2_test_score': array([0.8356889, 0.8356889, 0.8356889, 0.8356889, 0.8356889]),
  'split3_test_score': array([0.80628374, 0.80628374, 0.80628374, 0.80628374, 0.806

In [39]:
xgb2 = XGBClassifier(
    learning_rate=0.1,
    n_estimators=1000,
    max_depth=2,
    min_child_weight=0,
    gamma=0,
    subsample=0.8,
    colsample_bytree=0.8,
    objective= 'binary:logistic',
    nthread=4,
    scale_pos_weight=1,
    seed=27)
modelfit(xgb2, X_train, y_train, predictors)


Model Report
Accuracy : 0.9801
AUC Score (Train): 0.860247


In [42]:
xgb2.get_booster().best_iteration

242

In [43]:
param_test4 = {
    'subsample':[i/10.0 for i in range(6,10)],
    'colsample_bytree':[i/10.0 for i in range(6,10)]
}
gsearch4 = GridSearchCV(estimator = XGBClassifier(learning_rate=0.1, n_estimators=242, max_depth=2,
                                                  min_child_weight=0, gamma=0, subsample=0.8, colsample_bytree=0.8,
                                                  objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27),
                        param_grid = param_test4, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch4.fit(X_train[predictors],y_train)
gsearch4.cv_results_, gsearch4.best_params_, gsearch4.best_score_

({'mean_fit_time': array([ 92.93142452,  91.17673621,  89.64031339,  90.00686388,
          98.111092  ,  98.00340257,  99.80620499,  97.27631102,
         105.1011302 , 105.28546696, 101.22536883, 101.85385051,
         113.60227313, 111.05103765, 106.22310677, 108.43930783]),
  'std_fit_time': array([3.76722988, 2.44888148, 3.85982716, 4.55903728, 4.19814247,
         3.0352131 , 1.89645273, 3.97260387, 3.00438936, 3.4419448 ,
         2.89163417, 7.37751381, 3.72578925, 4.07584231, 3.59901584,
         9.58985861]),
  'mean_score_time': array([0.41181693, 0.38618369, 0.45879221, 0.39905281, 0.39406066,
         0.40343947, 0.42388282, 0.45350404, 0.44133611, 0.43435817,
         0.4235846 , 0.43146157, 0.43585291, 0.43415961, 0.43146248,
         0.32305942]),
  'std_score_time': array([0.03688218, 0.01341701, 0.03371429, 0.01979589, 0.03459696,
         0.01574551, 0.02255854, 0.03595569, 0.03006886, 0.03342054,
         0.01610972, 0.01022205, 0.05279756, 0.02778791, 0.04960381,
 

In [45]:
param_test5 = {
    'subsample':[i/100.0 for i in range(50,70,5)],
    'colsample_bytree':[i/100.0 for i in range(75,90,5)]
}
gsearch5 = GridSearchCV(estimator = XGBClassifier(learning_rate=0.1, n_estimators=242, max_depth=2,
                                                  min_child_weight=0, gamma=0, subsample=0.8, colsample_bytree=0.8,
                                                  objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27),
                        param_grid = param_test5, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch5.fit(X_train[predictors],y_train)
gsearch5.cv_results_, gsearch5.best_params_, gsearch5.best_score_

({'mean_fit_time': array([ 97.76801057,  98.56657963,  98.11280551,  99.954352  ,
         103.38891721, 101.74271789, 103.41593719, 103.54927011,
         104.87977915, 104.88900037, 107.39877548,  98.66346011]),
  'std_fit_time': array([2.89225536, 2.13124635, 3.64855221, 7.90027725, 3.8178794 ,
         3.89586674, 3.73453131, 6.00419252, 3.98998046, 3.95065081,
         3.43748374, 6.34675928]),
  'mean_score_time': array([0.39845157, 0.37770438, 0.39287057, 0.43096862, 0.43884149,
         0.39346538, 0.41779933, 0.42268534, 0.39615707, 0.4103179 ,
         0.40862184, 0.32165399]),
  'std_score_time': array([0.04565312, 0.03750591, 0.04833395, 0.03405274, 0.03769739,
         0.03960498, 0.05499106, 0.04481299, 0.0113437 , 0.03479261,
         0.03490197, 0.06603222]),
  'param_colsample_bytree': masked_array(data=[0.75, 0.75, 0.75, 0.75, 0.8, 0.8, 0.8, 0.8, 0.85, 0.85,
                     0.85, 0.85],
               mask=[False, False, False, False, False, False, False, False,


In [46]:
param_test6 = {
    'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100],
    'reg_lambda':[1e-5, 1e-2, 0.1, 1, 100]
}
gsearch6 = GridSearchCV(estimator = XGBClassifier(learning_rate=0.1, n_estimators=242, max_depth=2,
                                                  min_child_weight=0.5, gamma=0, subsample=0.65, colsample_bytree=0.85,
                                                  objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27),
                        param_grid = param_test6, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch6.fit(X_train[predictors],y_train)
gsearch6.cv_results_, gsearch6.best_params_, gsearch6.best_score_

({'mean_fit_time': array([102.82148795, 104.47865944, 105.03842044, 104.53564644,
         103.8814919 , 107.15806518, 104.6477572 , 105.08831959,
         104.17462635, 104.24339643, 103.13688703, 105.6539063 ,
         105.09010301, 107.61133561, 104.69984074, 107.19453812,
         107.0562469 , 113.47469687, 113.79648929, 114.79759498,
         100.25494986,  95.3439045 ,  94.29249077,  91.96366673,
          84.81702771]),
  'std_fit_time': array([ 5.90016353,  4.89108642,  3.94654116,  6.38416186,  6.55391335,
          4.62046716,  1.59895392,  1.86540298,  6.31867798,  1.92450061,
          3.1924875 ,  5.47429074,  7.64105478,  1.50017323,  1.0121701 ,
          5.87446152,  5.46939314,  2.3402783 ,  2.94489323,  2.42410405,
          2.67976846,  2.91436528,  3.12624587,  5.54358689, 16.23152392]),
  'mean_score_time': array([0.38957729, 0.41570625, 0.43535299, 0.39884953, 0.41361089,
         0.40623102, 0.41181588, 0.42328629, 0.38907695, 0.40284076,
         0.39775128, 0.

In [50]:
# import sys
# !{sys.executable} -m pip install featuretools

In [51]:
param_test7 = {
    'reg_alpha':[0.005, 0.01, 0.05],
    'reg_lambda':[90, 100, 110]
}
gsearch7 = GridSearchCV(estimator = XGBClassifier(learning_rate=0.1, n_estimators=242, max_depth=2,
                                                  min_child_weight=0, gamma=0, subsample=0.65, colsample_bytree=0.85,
                                                  objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27),
                        param_grid = param_test7, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch7.fit(X_train[predictors],y_train)
gsearch7.cv_results_, gsearch7.best_params_, gsearch7.best_score_

({'mean_fit_time': array([113.43013144, 107.46560378, 105.63868041, 109.90967846,
         108.98718281, 105.23187962, 105.18403721, 116.57514443,
         102.36544013]),
  'std_fit_time': array([ 2.47060544,  4.38806484,  4.22683558,  4.96470939,  1.96622374,
          5.70599849,  5.11804387,  4.18592093, 23.9876368 ]),
  'mean_score_time': array([0.42319379, 0.44004421, 0.42408705, 0.43176327, 0.43784766,
         0.4508162 , 0.43157039, 0.43195953, 0.38729644]),
  'std_score_time': array([0.04623584, 0.04339793, 0.03285735, 0.03480885, 0.02113793,
         0.02730139, 0.02727544, 0.03351534, 0.1074969 ]),
  'param_reg_alpha': masked_array(data=[0.005, 0.005, 0.005, 0.01, 0.01, 0.01, 0.05, 0.05,
                     0.05],
               mask=[False, False, False, False, False, False, False, False,
                     False],
         fill_value='?',
              dtype=object),
  'param_reg_lambda': masked_array(data=[90, 100, 110, 90, 100, 110, 90, 100, 110],
               mask

In [54]:
param_test8 = {
    'reg_lambda':[50,60,70,80,90]
}
gsearch8 = GridSearchCV(estimator = XGBClassifier(learning_rate=0.1, n_estimators=242, max_depth=2, reg_alpha=0.01,
                                                  min_child_weight=0, gamma=0, subsample=0.65, colsample_bytree=0.85,
                                                  objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27),
                        param_grid = param_test8, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch8.fit(X_train[predictors],y_train)
gsearch8.cv_results_, gsearch8.best_params_, gsearch8.best_score_

({'mean_fit_time': array([106.3192461 , 106.42303648, 105.45503507, 105.30041642,
          93.63567562]),
  'std_fit_time': array([ 2.7511178 ,  4.10503311,  7.54263332,  5.33956036, 22.27915578]),
  'mean_score_time': array([0.3853878 , 0.39057102, 0.44034452, 0.42548146, 0.34424491]),
  'std_score_time': array([0.02822538, 0.01212689, 0.01953183, 0.01861759, 0.10238654]),
  'param_reg_lambda': masked_array(data=[50, 60, 70, 80, 90],
               mask=[False, False, False, False, False],
         fill_value='?',
              dtype=object),
  'params': [{'reg_lambda': 50},
   {'reg_lambda': 60},
   {'reg_lambda': 70},
   {'reg_lambda': 80},
   {'reg_lambda': 90}],
  'split0_test_score': array([0.74100893, 0.74164445, 0.7401368 , 0.73996167, 0.74160969]),
  'split1_test_score': array([0.93281601, 0.93649815, 0.9362736 , 0.93503348, 0.93515663]),
  'split2_test_score': array([0.84107175, 0.83838922, 0.83904719, 0.83900338, 0.83961422]),
  'split3_test_score': array([0.81248278, 0.813

In [55]:
xgb3 = XGBClassifier(
    learning_rate=0.1,
    n_estimators=1000,
    max_depth=2,
    min_child_weight=0,
    gamma=0,
    subsample=0.65,
    colsample_bytree=0.85,
    reg_alpha=0.01,
    reg_lambda=60,
    objective= 'binary:logistic',
    nthread=4,
    scale_pos_weight=1,
    seed=27)
modelfit(xgb3, X_train, y_train, predictors)


Model Report
Accuracy : 0.9800
AUC Score (Train): 0.863203


In [58]:
xgb3.get_booster().best_iteration

430

In [59]:
param_test9 = {
    'learning_rate':[0.001,0.01,0.1,1,2,3]
}
gsearch9 = GridSearchCV(estimator = XGBClassifier(learning_rate=0.1, n_estimators=430, max_depth=2, reg_alpha=0.01,
                                                  reg_lambda=60, min_child_weight=0, gamma=0, subsample=0.65, 
                                                  colsample_bytree=0.85, objective= 'binary:logistic', nthread=4, 
                                                  scale_pos_weight=1,seed=27),
                        param_grid = param_test9, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch9.fit(X_train[predictors],y_train)
gsearch9.cv_results_, gsearch9.best_params_, gsearch9.best_score_

({'mean_fit_time': array([189.57319698, 185.0477819 , 187.50591111, 187.65393124,
         188.74061122, 161.39826283]),
  'std_fit_time': array([ 5.37642374,  9.47854844,  8.21348573,  3.85987616,  5.7678976 ,
         29.3092392 ]),
  'mean_score_time': array([0.48052931, 0.44063687, 0.44562631, 0.48632216, 0.45001612,
         0.34650464]),
  'std_score_time': array([0.06222679, 0.03194171, 0.03314176, 0.05796995, 0.03582105,
         0.07760076]),
  'param_learning_rate': masked_array(data=[0.001, 0.01, 0.1, 1, 2, 3],
               mask=[False, False, False, False, False, False],
         fill_value='?',
              dtype=object),
  'params': [{'learning_rate': 0.001},
   {'learning_rate': 0.01},
   {'learning_rate': 0.1},
   {'learning_rate': 1},
   {'learning_rate': 2},
   {'learning_rate': 3}],
  'split0_test_score': array([0.68314107, 0.72632672, 0.74467207, 0.71455721, 0.69787321,
         0.59099114]),
  'split1_test_score': array([0.5159176 , 0.94956728, 0.93039235, 0.862

In [62]:
param_test10 = {
    'learning_rate':[0.05,0.1,0.2,0.3]
}
gsearch10 = GridSearchCV(estimator = XGBClassifier(learning_rate=0.1, n_estimators=430, max_depth=2, reg_alpha=0.01,
                                                  reg_lambda=60, min_child_weight=0, gamma=0, subsample=0.65, 
                                                  colsample_bytree=0.85, objective= 'binary:logistic', nthread=4, 
                                                  scale_pos_weight=1,seed=27),
                        param_grid = param_test10, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch10.fit(X_train[predictors],y_train)
gsearch10.cv_results_, gsearch10.best_params_, gsearch10.best_score_

({'mean_fit_time': array([186.0502316 , 183.98697796, 183.34372196, 174.76897497]),
  'std_fit_time': array([ 5.29128131, 14.46704649, 10.77241369, 10.79772728]),
  'mean_score_time': array([0.44772382, 0.44503279, 0.45310888, 0.36658511]),
  'std_score_time': array([0.05299814, 0.01338763, 0.02575325, 0.09779237]),
  'param_learning_rate': masked_array(data=[0.05, 0.1, 0.2, 0.3],
               mask=[False, False, False, False],
         fill_value='?',
              dtype=object),
  'params': [{'learning_rate': 0.05},
   {'learning_rate': 0.1},
   {'learning_rate': 0.2},
   {'learning_rate': 0.3}],
  'split0_test_score': array([0.73900644, 0.74467207, 0.74565842, 0.74384764]),
  'split1_test_score': array([0.93172296, 0.93039235, 0.91950026, 0.90274541]),
  'split2_test_score': array([0.83802111, 0.83450746, 0.8285692 , 0.82144853]),
  'split3_test_score': array([0.81098642, 0.81345032, 0.80970402, 0.80458393]),
  'split4_test_score': array([0.81124758, 0.81179821, 0.80725663, 0.8087

In [63]:
xgb4 = XGBClassifier(
    learning_rate=0.1,
    n_estimators=1000,
    max_depth=2,
    min_child_weight=0,
    gamma=0,
    subsample=0.65,
    colsample_bytree=0.85,
    reg_alpha=0.01,
    reg_lambda=60,
    objective= 'binary:logistic',
    nthread=4,
    scale_pos_weight=1,
    seed=27)
modelfit(xgb4, X_train, y_train, predictors)


Model Report
Accuracy : 0.9800
AUC Score (Train): 0.863203


stacking

In [31]:
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import OneHotEncoder

def auc(y_true, y_pred):
    """ROC AUC metric for both binary and multiclass classification.
    
    Parameters
    ----------
    y_true : 1d numpy array
        True class labels
    y_pred : 2d numpy array
        Predicted probabilities for each class
    """
    ohe = OneHotEncoder(categories='auto',sparse=False)
    y_true = ohe.fit_transform(y_true.reshape(-1, 1))
    auc_score = roc_auc_score(y_true, y_pred)
    return auc_score

In [32]:
models = [
    GaussianNB(),
    
    LogisticRegression(random_state=27),
    
    CatBoostClassifier(random_state=27, depth=8, 
                       iterations=622, l2_leaf_reg=1, learning_rate=0.5401606783724958),

    ExtraTreesClassifier(random_state=27, n_jobs=-1, 
                         n_estimators=180, max_depth=5),
            
    RandomForestClassifier(random_state=27, n_jobs=-1, 
                           n_estimators=1000, max_depth=180),
        
    XGBClassifier(random_state=27, n_jobs=-1, learning_rate=0.01,
                  n_estimators=1000, max_depth=2, min_child_weight=0.5,
                  gamma=0, subsample=0.6, colsample_bytree=0.55,
                  reg_alpha=0.01, reg_lambda=100, objective= 'binary:logistic',
                  nthread=4, scale_pos_weight=1, seed=27),
    
    LGBMClassifier(random_state=27, n_jobs=-1, boosting_type="gbdt", learning_rate=0.01, n_estimators=420, 
                   num_leaves=4, is_unbalance=True, max_depth=8, feature_fraction=0.5, bagging_fraction=0.8, bagging_freq=15)
]

In [33]:
S_train, S_test = stacking(models,                   
                           X_train[predictors], y_train, X_test[predictors],   # our data
                           regression=False, 
#                            Boolean indicating whether we want to use the function for regression. 
#                            In our case set to False since this is a classification
                           mode='oof_pred_bag', 
#                            using the earlier describe out-of-fold during cross-validation
                           needs_proba=True,
         
                           save_dir=None, 
#                            save the result to directory Boolean
                           metric=auc, 
#                            what evaluation metric to use (we imported the accuracy_score in the beginning)    
                           n_folds=5, 
#                            how many folds to use for cross-validation
                           stratified=True,
#                            whether to use stratified cross-validation            
                           shuffle=True, 
#                            whether to shuffle the data            
                           random_state=0,    
         
                           verbose=2 
#                           2 here refers to printing all info
                          )

task:         [classification]
n_classes:    [2]
metric:       [auc]
mode:         [oof_pred_bag]
n_models:     [7]

model  0:     [GaussianNB]
    fold  0:  [0.72637911]
    fold  1:  [0.72379062]
    fold  2:  [0.73593960]
    fold  3:  [0.71751770]
    fold  4:  [0.72627018]
    ----
    MEAN:     [0.72597945] + [0.00593028]
    FULL:     [0.72327601]

model  1:     [LogisticRegression]
    fold  0:  [0.84930829]
    fold  1:  [0.83411378]
    fold  2:  [0.81999949]
    fold  3:  [0.81512717]
    fold  4:  [0.83300204]
    ----
    MEAN:     [0.83031015] + [0.01199563]
    FULL:     [0.83014607]

model  2:     [CatBoostClassifier]
0:	learn: 0.1450977	total: 57.8ms	remaining: 35.9s
1:	learn: 0.0947693	total: 124ms	remaining: 38.3s
2:	learn: 0.0844949	total: 189ms	remaining: 39s
3:	learn: 0.0823732	total: 252ms	remaining: 38.9s
4:	learn: 0.0802110	total: 320ms	remaining: 39.5s
5:	learn: 0.0794942	total: 381ms	remaining: 39.2s
6:	learn: 0.0789219	total: 444ms	remaining: 39s
7:	learn: 0

151:	learn: 0.0251029	total: 9.82s	remaining: 30.4s
152:	learn: 0.0248072	total: 9.88s	remaining: 30.3s
153:	learn: 0.0246905	total: 9.95s	remaining: 30.2s
154:	learn: 0.0241774	total: 10s	remaining: 30.2s
155:	learn: 0.0239900	total: 10.1s	remaining: 30.1s
156:	learn: 0.0237273	total: 10.1s	remaining: 30s
157:	learn: 0.0234660	total: 10.2s	remaining: 30s
158:	learn: 0.0233560	total: 10.3s	remaining: 29.9s
159:	learn: 0.0230920	total: 10.3s	remaining: 29.9s
160:	learn: 0.0228380	total: 10.4s	remaining: 29.8s
161:	learn: 0.0225724	total: 10.5s	remaining: 29.7s
162:	learn: 0.0222102	total: 10.5s	remaining: 29.7s
163:	learn: 0.0220489	total: 10.6s	remaining: 29.6s
164:	learn: 0.0218554	total: 10.7s	remaining: 29.6s
165:	learn: 0.0217541	total: 10.7s	remaining: 29.5s
166:	learn: 0.0215772	total: 10.8s	remaining: 29.4s
167:	learn: 0.0213028	total: 10.9s	remaining: 29.4s
168:	learn: 0.0211087	total: 10.9s	remaining: 29.3s
169:	learn: 0.0207735	total: 11s	remaining: 29.2s
170:	learn: 0.020671

310:	learn: 0.0056989	total: 20.7s	remaining: 20.7s
311:	learn: 0.0056779	total: 20.8s	remaining: 20.7s
312:	learn: 0.0056095	total: 20.9s	remaining: 20.6s
313:	learn: 0.0055391	total: 20.9s	remaining: 20.5s
314:	learn: 0.0054912	total: 21s	remaining: 20.5s
315:	learn: 0.0054522	total: 21.1s	remaining: 20.4s
316:	learn: 0.0054114	total: 21.1s	remaining: 20.3s
317:	learn: 0.0053580	total: 21.2s	remaining: 20.3s
318:	learn: 0.0053211	total: 21.3s	remaining: 20.2s
319:	learn: 0.0052693	total: 21.3s	remaining: 20.1s
320:	learn: 0.0052286	total: 21.4s	remaining: 20.1s
321:	learn: 0.0051325	total: 21.5s	remaining: 20s
322:	learn: 0.0050848	total: 21.5s	remaining: 19.9s
323:	learn: 0.0050487	total: 21.6s	remaining: 19.9s
324:	learn: 0.0050292	total: 21.7s	remaining: 19.8s
325:	learn: 0.0049745	total: 21.7s	remaining: 19.7s
326:	learn: 0.0048754	total: 21.8s	remaining: 19.7s
327:	learn: 0.0048393	total: 21.9s	remaining: 19.6s
328:	learn: 0.0048217	total: 21.9s	remaining: 19.5s
329:	learn: 0.00

472:	learn: 0.0018071	total: 31.6s	remaining: 9.94s
473:	learn: 0.0017856	total: 31.6s	remaining: 9.87s
474:	learn: 0.0017754	total: 31.7s	remaining: 9.8s
475:	learn: 0.0017653	total: 31.8s	remaining: 9.74s
476:	learn: 0.0017580	total: 31.8s	remaining: 9.67s
477:	learn: 0.0017534	total: 31.9s	remaining: 9.6s
478:	learn: 0.0017413	total: 31.9s	remaining: 9.54s
479:	learn: 0.0017278	total: 32s	remaining: 9.47s
480:	learn: 0.0017215	total: 32.1s	remaining: 9.4s
481:	learn: 0.0017103	total: 32.1s	remaining: 9.34s
482:	learn: 0.0017003	total: 32.2s	remaining: 9.27s
483:	learn: 0.0016962	total: 32.3s	remaining: 9.2s
484:	learn: 0.0016898	total: 32.3s	remaining: 9.14s
485:	learn: 0.0016840	total: 32.4s	remaining: 9.07s
486:	learn: 0.0016738	total: 32.5s	remaining: 9s
487:	learn: 0.0016700	total: 32.5s	remaining: 8.93s
488:	learn: 0.0016605	total: 32.6s	remaining: 8.87s
489:	learn: 0.0016549	total: 32.7s	remaining: 8.8s
490:	learn: 0.0016506	total: 32.7s	remaining: 8.73s
491:	learn: 0.0016409	

10:	learn: 0.0736337	total: 675ms	remaining: 37.5s
11:	learn: 0.0728639	total: 737ms	remaining: 37.5s
12:	learn: 0.0724830	total: 797ms	remaining: 37.3s
13:	learn: 0.0712391	total: 864ms	remaining: 37.5s
14:	learn: 0.0705128	total: 926ms	remaining: 37.5s
15:	learn: 0.0700880	total: 988ms	remaining: 37.4s
16:	learn: 0.0692379	total: 1.05s	remaining: 37.4s
17:	learn: 0.0689886	total: 1.11s	remaining: 37.3s
18:	learn: 0.0685730	total: 1.17s	remaining: 37.2s
19:	learn: 0.0680917	total: 1.23s	remaining: 37.1s
20:	learn: 0.0671974	total: 1.3s	remaining: 37.1s
21:	learn: 0.0664812	total: 1.36s	remaining: 37.1s
22:	learn: 0.0660794	total: 1.42s	remaining: 37s
23:	learn: 0.0656604	total: 1.48s	remaining: 36.9s
24:	learn: 0.0650470	total: 1.54s	remaining: 36.8s
25:	learn: 0.0647367	total: 1.6s	remaining: 36.7s
26:	learn: 0.0643137	total: 1.66s	remaining: 36.6s
27:	learn: 0.0636180	total: 1.73s	remaining: 36.7s
28:	learn: 0.0632393	total: 1.79s	remaining: 36.6s
29:	learn: 0.0623114	total: 1.85s	r

...

532:	learn: 0.0013627	total: 35.4s	remaining: 5.91s
533:	learn: 0.0013527	total: 35.5s	remaining: 5.85s
534:	learn: 0.0013380	total: 35.6s	remaining: 5.78s
535:	learn: 0.0013368	total: 35.6s	remaining: 5.71s
536:	learn: 0.0013336	total: 35.7s	remaining: 5.65s
537:	learn: 0.0013300	total: 35.7s	remaining: 5.58s
538:	learn: 0.0013259	total: 35.8s	remaining: 5.51s
539:	learn: 0.0013205	total: 35.9s	remaining: 5.45s
540:	learn: 0.0013094	total: 35.9s	remaining: 5.38s
541:	learn: 0.0013012	total: 36s	remaining: 5.32s
542:	learn: 0.0012910	total: 36.1s	remaining: 5.25s
543:	learn: 0.0012829	total: 36.1s	remaining: 5.18s
544:	learn: 0.0012767	total: 36.2s	remaining: 5.12s
545:	learn: 0.0012725	total: 36.3s	remaining: 5.05s
546:	learn: 0.0012688	total: 36.3s	remaining: 4.98s
547:	learn: 0.0012624	total: 36.4s	remaining: 4.92s
548:	learn: 0.0012557	total: 36.5s	remaining: 4.85s
549:	learn: 0.0012476	total: 36.5s	remaining: 4.78s
550:	learn: 0.0012445	total: 36.6s	remaining: 4.72s
551:	learn: 0.

72:	learn: 0.0462680	total: 4.56s	remaining: 34.3s
73:	learn: 0.0459237	total: 4.63s	remaining: 34.3s
74:	learn: 0.0456604	total: 4.69s	remaining: 34.2s
75:	learn: 0.0452037	total: 4.75s	remaining: 34.2s
76:	learn: 0.0449368	total: 4.82s	remaining: 34.1s
77:	learn: 0.0446062	total: 4.88s	remaining: 34s
78:	learn: 0.0442246	total: 4.94s	remaining: 34s
79:	learn: 0.0434793	total: 5s	remaining: 33.9s
80:	learn: 0.0431679	total: 5.07s	remaining: 33.9s
81:	learn: 0.0428938	total: 5.14s	remaining: 33.8s
82:	learn: 0.0426206	total: 5.2s	remaining: 33.8s
83:	learn: 0.0424076	total: 5.26s	remaining: 33.7s
84:	learn: 0.0420604	total: 5.33s	remaining: 33.7s
85:	learn: 0.0418591	total: 5.39s	remaining: 33.6s
86:	learn: 0.0414960	total: 5.45s	remaining: 33.5s
87:	learn: 0.0412925	total: 5.52s	remaining: 33.5s
88:	learn: 0.0406783	total: 5.58s	remaining: 33.4s
89:	learn: 0.0404252	total: 5.64s	remaining: 33.3s
90:	learn: 0.0399396	total: 5.71s	remaining: 33.3s
91:	learn: 0.0396220	total: 5.77s	remai

234:	learn: 0.0106028	total: 14.9s	remaining: 24.6s
235:	learn: 0.0104031	total: 15s	remaining: 24.6s
236:	learn: 0.0102696	total: 15.1s	remaining: 24.5s
237:	learn: 0.0101809	total: 15.1s	remaining: 24.4s
238:	learn: 0.0100634	total: 15.2s	remaining: 24.4s
239:	learn: 0.0100068	total: 15.3s	remaining: 24.3s
240:	learn: 0.0099252	total: 15.3s	remaining: 24.2s
241:	learn: 0.0098595	total: 15.4s	remaining: 24.2s
242:	learn: 0.0098379	total: 15.5s	remaining: 24.1s
243:	learn: 0.0096791	total: 15.5s	remaining: 24s
244:	learn: 0.0096516	total: 15.6s	remaining: 24s
245:	learn: 0.0095667	total: 15.6s	remaining: 23.9s
246:	learn: 0.0094578	total: 15.7s	remaining: 23.9s
247:	learn: 0.0093421	total: 15.8s	remaining: 23.8s
248:	learn: 0.0093028	total: 15.8s	remaining: 23.7s
249:	learn: 0.0092643	total: 15.9s	remaining: 23.7s
250:	learn: 0.0092262	total: 16s	remaining: 23.6s
251:	learn: 0.0091338	total: 16s	remaining: 23.6s
252:	learn: 0.0090278	total: 16.1s	remaining: 23.5s
253:	learn: 0.0089784	

395:	learn: 0.0029083	total: 25.6s	remaining: 14.6s
396:	learn: 0.0028728	total: 25.7s	remaining: 14.6s
397:	learn: 0.0028497	total: 25.8s	remaining: 14.5s
398:	learn: 0.0028322	total: 25.9s	remaining: 14.5s
399:	learn: 0.0028017	total: 25.9s	remaining: 14.4s
400:	learn: 0.0027579	total: 26s	remaining: 14.3s
401:	learn: 0.0027487	total: 26.1s	remaining: 14.3s
402:	learn: 0.0027359	total: 26.1s	remaining: 14.2s
403:	learn: 0.0027258	total: 26.2s	remaining: 14.1s
404:	learn: 0.0027114	total: 26.3s	remaining: 14.1s
405:	learn: 0.0026999	total: 26.3s	remaining: 14s
406:	learn: 0.0026970	total: 26.4s	remaining: 13.9s
407:	learn: 0.0026539	total: 26.4s	remaining: 13.9s
408:	learn: 0.0026369	total: 26.5s	remaining: 13.8s
409:	learn: 0.0026139	total: 26.6s	remaining: 13.7s
410:	learn: 0.0025968	total: 26.7s	remaining: 13.7s
411:	learn: 0.0025862	total: 26.7s	remaining: 13.6s
412:	learn: 0.0025822	total: 26.8s	remaining: 13.6s
413:	learn: 0.0025521	total: 26.9s	remaining: 13.5s
414:	learn: 0.00

554:	learn: 0.0011969	total: 36.7s	remaining: 4.43s
555:	learn: 0.0011919	total: 36.8s	remaining: 4.37s
556:	learn: 0.0011887	total: 36.9s	remaining: 4.3s
557:	learn: 0.0011853	total: 36.9s	remaining: 4.24s
558:	learn: 0.0011812	total: 37s	remaining: 4.17s
559:	learn: 0.0011758	total: 37.1s	remaining: 4.1s
560:	learn: 0.0011733	total: 37.1s	remaining: 4.04s
561:	learn: 0.0011620	total: 37.2s	remaining: 3.97s
562:	learn: 0.0011494	total: 37.3s	remaining: 3.91s
563:	learn: 0.0011466	total: 37.4s	remaining: 3.84s
564:	learn: 0.0011433	total: 37.4s	remaining: 3.78s
565:	learn: 0.0011370	total: 37.5s	remaining: 3.71s
566:	learn: 0.0011269	total: 37.6s	remaining: 3.65s
567:	learn: 0.0011239	total: 37.7s	remaining: 3.58s
568:	learn: 0.0011193	total: 37.7s	remaining: 3.51s
569:	learn: 0.0011175	total: 37.8s	remaining: 3.45s
570:	learn: 0.0011161	total: 37.9s	remaining: 3.38s
571:	learn: 0.0011040	total: 37.9s	remaining: 3.32s
572:	learn: 0.0010993	total: 38s	remaining: 3.25s
573:	learn: 0.0010

In [34]:
model = XGBClassifier(random_state=0, n_jobs=-1, learning_rate=0.01,
                  n_estimators=1000, max_depth=2, min_child_weight=0.5,
                  gamma=0, subsample=0.6, colsample_bytree=0.55,
                  reg_alpha=0.01, reg_lambda=100, objective= 'binary:logistic',
                  nthread=4, scale_pos_weight=1, seed=27)
model = model.fit(S_train, y_train)
y_pred = model.predict_proba(S_test)
y_pred

array([[0.92821604, 0.07178395],
       [0.9980326 , 0.00196734],
       [0.9948278 , 0.00517221],
       ...,
       [0.77435523, 0.22564477],
       [0.7765839 , 0.2234161 ],
       [0.8782844 , 0.12171561]], dtype=float32)

In [40]:
pd.DataFrame(y_pred).to_csv("stacking_try_1.csv")

In [38]:
S_train.shape, S_test.shape

((100000, 14), (150000, 14))

In [39]:
modelss = [
    GaussianNB(),
    
    LogisticRegression(random_state=27),
    
    CatBoostClassifier(random_state=27, depth=10, 
                       iterations=622, l2_leaf_reg=10, learning_rate=0.5401606783724958),

    ExtraTreesClassifier(random_state=27, n_jobs=-1, 
                         n_estimators=180, max_depth=5),
            
    RandomForestClassifier(random_state=27, n_jobs=-1, 
                           n_estimators=1000, max_depth=180),
        
    XGBClassifier(random_state=27, n_jobs=-1, learning_rate=0.01,
                  n_estimators=1000, max_depth=2, min_child_weight=0.5,
                  gamma=0, subsample=0.6, colsample_bytree=0.55,
                  reg_alpha=0.01, reg_lambda=100, objective= 'binary:logistic',
                  nthread=4, scale_pos_weight=1, seed=27),
    
    LGBMClassifier(random_state=27, n_jobs=-1, boosting_type="gbdt", learning_rate=0.01, n_estimators=420, 
                   num_leaves=4, is_unbalance=True, max_depth=8, feature_fraction=0.5, bagging_fraction=0.8, bagging_freq=15)
]

In [40]:
S_train1, S_test1 = stacking(modelss,                   
                           X_train[predictors], y_train, X_test[predictors],   # our data
                           regression=False, 
#                            Boolean indicating whether we want to use the function for regression. 
#                            In our case set to False since this is a classification
                           mode='oof_pred_bag', 
#                            using the earlier describe out-of-fold during cross-validation
                           needs_proba=True,
         
                           save_dir=None, 
#                            save the result to directory Boolean
                           metric=auc, 
#                            what evaluation metric to use (we imported the accuracy_score in the beginning)    
                           n_folds=5, 
#                            how many folds to use for cross-validation
                           stratified=True,
#                            whether to use stratified cross-validation            
                           shuffle=True, 
#                            whether to shuffle the data            
                           random_state=0,    
         
                           verbose=2 
#                           2 here refers to printing all info
                          )

task:         [classification]
n_classes:    [2]
metric:       [auc]
mode:         [oof_pred_bag]
n_models:     [7]

model  0:     [GaussianNB]
    fold  0:  [0.72637911]
    fold  1:  [0.72379062]
    fold  2:  [0.73593960]
    fold  3:  [0.71751770]
    fold  4:  [0.72627018]
    ----
    MEAN:     [0.72597945] + [0.00593028]
    FULL:     [0.72327601]

model  1:     [LogisticRegression]
    fold  0:  [0.84930829]
    fold  1:  [0.83411378]
    fold  2:  [0.81999949]
    fold  3:  [0.81512717]
    fold  4:  [0.83300204]
    ----
    MEAN:     [0.83031015] + [0.01199563]
    FULL:     [0.83014607]

model  2:     [CatBoostClassifier]
0:	learn: 0.1481872	total: 157ms	remaining: 1m 37s
1:	learn: 0.0964825	total: 309ms	remaining: 1m 35s
2:	learn: 0.0846751	total: 458ms	remaining: 1m 34s
3:	learn: 0.0807627	total: 644ms	remaining: 1m 39s
4:	learn: 0.0796370	total: 841ms	remaining: 1m 43s
5:	learn: 0.0763448	total: 1s	remaining: 1m 42s
6:	learn: 0.0753036	total: 1.15s	remaining: 1m 41s
7:	l

146:	learn: 0.0277471	total: 24.4s	remaining: 1m 18s
147:	learn: 0.0276424	total: 24.5s	remaining: 1m 18s
148:	learn: 0.0274188	total: 24.7s	remaining: 1m 18s
149:	learn: 0.0273078	total: 24.8s	remaining: 1m 18s
150:	learn: 0.0270045	total: 25s	remaining: 1m 17s
151:	learn: 0.0267708	total: 25.1s	remaining: 1m 17s
152:	learn: 0.0265648	total: 25.3s	remaining: 1m 17s
153:	learn: 0.0262010	total: 25.5s	remaining: 1m 17s
154:	learn: 0.0258807	total: 25.6s	remaining: 1m 17s
155:	learn: 0.0258087	total: 25.8s	remaining: 1m 16s
156:	learn: 0.0254811	total: 25.9s	remaining: 1m 16s
157:	learn: 0.0253159	total: 26.1s	remaining: 1m 16s
158:	learn: 0.0252056	total: 26.2s	remaining: 1m 16s
159:	learn: 0.0246820	total: 26.4s	remaining: 1m 16s
160:	learn: 0.0246258	total: 26.5s	remaining: 1m 16s
161:	learn: 0.0243911	total: 26.7s	remaining: 1m 15s
162:	learn: 0.0242654	total: 26.9s	remaining: 1m 15s
163:	learn: 0.0242108	total: 27s	remaining: 1m 15s
164:	learn: 0.0236143	total: 27.2s	remaining: 1m 1

304:	learn: 0.0086439	total: 48.6s	remaining: 50.5s
305:	learn: 0.0086072	total: 48.7s	remaining: 50.3s
306:	learn: 0.0085681	total: 48.9s	remaining: 50.2s
307:	learn: 0.0085583	total: 49s	remaining: 50s
308:	learn: 0.0085480	total: 49.2s	remaining: 49.8s
309:	learn: 0.0083563	total: 49.4s	remaining: 49.7s
310:	learn: 0.0083354	total: 49.5s	remaining: 49.5s
311:	learn: 0.0081974	total: 49.7s	remaining: 49.3s
312:	learn: 0.0081711	total: 49.8s	remaining: 49.2s
313:	learn: 0.0081547	total: 50s	remaining: 49s
314:	learn: 0.0081326	total: 50.1s	remaining: 48.8s
315:	learn: 0.0081126	total: 50.3s	remaining: 48.7s
316:	learn: 0.0081094	total: 50.4s	remaining: 48.5s
317:	learn: 0.0080629	total: 50.6s	remaining: 48.3s
318:	learn: 0.0080588	total: 50.7s	remaining: 48.2s
319:	learn: 0.0080096	total: 50.9s	remaining: 48s
320:	learn: 0.0079919	total: 51s	remaining: 47.8s
321:	learn: 0.0079582	total: 51.2s	remaining: 47.7s
322:	learn: 0.0079236	total: 51.3s	remaining: 47.5s
323:	learn: 0.0078642	to

464:	learn: 0.0042339	total: 1m 14s	remaining: 25.1s
465:	learn: 0.0042134	total: 1m 14s	remaining: 24.9s
466:	learn: 0.0041906	total: 1m 14s	remaining: 24.8s
467:	learn: 0.0041839	total: 1m 14s	remaining: 24.6s
468:	learn: 0.0041691	total: 1m 14s	remaining: 24.5s
469:	learn: 0.0041558	total: 1m 15s	remaining: 24.3s
470:	learn: 0.0041357	total: 1m 15s	remaining: 24.1s
471:	learn: 0.0041210	total: 1m 15s	remaining: 24s
472:	learn: 0.0041159	total: 1m 15s	remaining: 23.8s
473:	learn: 0.0040668	total: 1m 15s	remaining: 23.7s
474:	learn: 0.0040273	total: 1m 15s	remaining: 23.5s
475:	learn: 0.0040174	total: 1m 16s	remaining: 23.3s
476:	learn: 0.0039977	total: 1m 16s	remaining: 23.2s
477:	learn: 0.0039954	total: 1m 16s	remaining: 23s
478:	learn: 0.0039787	total: 1m 16s	remaining: 22.8s
479:	learn: 0.0039275	total: 1m 16s	remaining: 22.7s
480:	learn: 0.0039187	total: 1m 16s	remaining: 22.5s
481:	learn: 0.0039182	total: 1m 16s	remaining: 22.4s
482:	learn: 0.0039099	total: 1m 17s	remaining: 22.

621:	learn: 0.0023906	total: 1m 40s	remaining: 0us
    fold  0:  [0.78120115]
0:	learn: 0.1456643	total: 151ms	remaining: 1m 33s
1:	learn: 0.0922352	total: 298ms	remaining: 1m 32s
2:	learn: 0.0843709	total: 478ms	remaining: 1m 38s
3:	learn: 0.0801130	total: 637ms	remaining: 1m 38s
4:	learn: 0.0775622	total: 803ms	remaining: 1m 39s
5:	learn: 0.0774802	total: 986ms	remaining: 1m 41s
6:	learn: 0.0773881	total: 1.14s	remaining: 1m 40s
7:	learn: 0.0763124	total: 1.3s	remaining: 1m 39s
8:	learn: 0.0758458	total: 1.46s	remaining: 1m 39s
9:	learn: 0.0751103	total: 1.62s	remaining: 1m 39s
10:	learn: 0.0732165	total: 1.79s	remaining: 1m 39s
11:	learn: 0.0721902	total: 1.94s	remaining: 1m 38s
12:	learn: 0.0714411	total: 2.11s	remaining: 1m 38s
13:	learn: 0.0707149	total: 2.31s	remaining: 1m 40s
14:	learn: 0.0701730	total: 2.51s	remaining: 1m 41s
15:	learn: 0.0697846	total: 2.71s	remaining: 1m 42s
16:	learn: 0.0695711	total: 2.88s	remaining: 1m 42s
17:	learn: 0.0693334	total: 3.04s	remaining: 1m 4

...

489:	learn: 0.0036967	total: 1m 17s	remaining: 20.9s
490:	learn: 0.0036799	total: 1m 17s	remaining: 20.8s
491:	learn: 0.0036796	total: 1m 18s	remaining: 20.6s
492:	learn: 0.0036649	total: 1m 18s	remaining: 20.5s
493:	learn: 0.0036618	total: 1m 18s	remaining: 20.3s
494:	learn: 0.0036440	total: 1m 18s	remaining: 20.1s
495:	learn: 0.0036320	total: 1m 18s	remaining: 20s
496:	learn: 0.0036211	total: 1m 18s	remaining: 19.8s
497:	learn: 0.0036026	total: 1m 19s	remaining: 19.7s
498:	learn: 0.0035947	total: 1m 19s	remaining: 19.5s
499:	learn: 0.0035887	total: 1m 19s	remaining: 19.4s
500:	learn: 0.0035700	total: 1m 19s	remaining: 19.2s
501:	learn: 0.0035388	total: 1m 19s	remaining: 19s
502:	learn: 0.0035203	total: 1m 19s	remaining: 18.9s
503:	learn: 0.0035010	total: 1m 19s	remaining: 18.7s
504:	learn: 0.0034934	total: 1m 20s	remaining: 18.6s
505:	learn: 0.0034775	total: 1m 20s	remaining: 18.4s
506:	learn: 0.0034510	total: 1m 20s	remaining: 18.2s
507:	learn: 0.0034466	total: 1m 20s	remaining: 18.

24:	learn: 0.0632437	total: 3.96s	remaining: 1m 34s
25:	learn: 0.0630093	total: 4.11s	remaining: 1m 34s
26:	learn: 0.0624505	total: 4.27s	remaining: 1m 34s
27:	learn: 0.0621687	total: 4.42s	remaining: 1m 33s
28:	learn: 0.0621011	total: 4.59s	remaining: 1m 33s
29:	learn: 0.0616421	total: 4.76s	remaining: 1m 34s
30:	learn: 0.0611652	total: 4.98s	remaining: 1m 34s
31:	learn: 0.0605773	total: 5.18s	remaining: 1m 35s
32:	learn: 0.0602735	total: 5.39s	remaining: 1m 36s
33:	learn: 0.0601485	total: 5.56s	remaining: 1m 36s
34:	learn: 0.0597697	total: 5.73s	remaining: 1m 36s
35:	learn: 0.0594196	total: 5.89s	remaining: 1m 35s
36:	learn: 0.0592348	total: 6.06s	remaining: 1m 35s
37:	learn: 0.0591489	total: 6.24s	remaining: 1m 35s
38:	learn: 0.0585284	total: 6.39s	remaining: 1m 35s
39:	learn: 0.0583450	total: 6.6s	remaining: 1m 36s
40:	learn: 0.0582066	total: 6.78s	remaining: 1m 36s
41:	learn: 0.0574948	total: 6.95s	remaining: 1m 35s
42:	learn: 0.0566879	total: 7.17s	remaining: 1m 36s
43:	learn: 0.

182:	learn: 0.0226183	total: 30.9s	remaining: 1m 14s
183:	learn: 0.0224249	total: 31s	remaining: 1m 13s
184:	learn: 0.0224001	total: 31.2s	remaining: 1m 13s
185:	learn: 0.0222504	total: 31.4s	remaining: 1m 13s
186:	learn: 0.0222159	total: 31.5s	remaining: 1m 13s
187:	learn: 0.0221627	total: 31.7s	remaining: 1m 13s
188:	learn: 0.0221172	total: 31.9s	remaining: 1m 12s
189:	learn: 0.0220808	total: 32s	remaining: 1m 12s
190:	learn: 0.0218603	total: 32.2s	remaining: 1m 12s
191:	learn: 0.0215304	total: 32.4s	remaining: 1m 12s
192:	learn: 0.0214176	total: 32.5s	remaining: 1m 12s
193:	learn: 0.0211485	total: 32.7s	remaining: 1m 12s
194:	learn: 0.0211076	total: 32.8s	remaining: 1m 11s
195:	learn: 0.0209181	total: 33s	remaining: 1m 11s
196:	learn: 0.0208280	total: 33.2s	remaining: 1m 11s
197:	learn: 0.0207588	total: 33.3s	remaining: 1m 11s
198:	learn: 0.0204504	total: 33.5s	remaining: 1m 11s
199:	learn: 0.0203171	total: 33.6s	remaining: 1m 10s
200:	learn: 0.0203096	total: 33.8s	remaining: 1m 10s

341:	learn: 0.0085737	total: 58.6s	remaining: 48s
342:	learn: 0.0084893	total: 58.8s	remaining: 47.8s
343:	learn: 0.0083283	total: 59s	remaining: 47.7s
344:	learn: 0.0082865	total: 59.1s	remaining: 47.5s
345:	learn: 0.0082825	total: 59.3s	remaining: 47.3s
346:	learn: 0.0082659	total: 59.5s	remaining: 47.1s
347:	learn: 0.0082329	total: 59.6s	remaining: 47s
348:	learn: 0.0081769	total: 59.8s	remaining: 46.8s
349:	learn: 0.0081411	total: 1m	remaining: 46.6s
350:	learn: 0.0080187	total: 1m	remaining: 46.5s
351:	learn: 0.0079521	total: 1m	remaining: 46.3s
352:	learn: 0.0079236	total: 1m	remaining: 46.1s
353:	learn: 0.0078274	total: 1m	remaining: 46s
354:	learn: 0.0078231	total: 1m	remaining: 45.8s
355:	learn: 0.0077516	total: 1m 1s	remaining: 45.6s
356:	learn: 0.0076687	total: 1m 1s	remaining: 45.4s
357:	learn: 0.0076661	total: 1m 1s	remaining: 45.3s
358:	learn: 0.0076155	total: 1m 1s	remaining: 45.1s
359:	learn: 0.0075196	total: 1m 1s	remaining: 45s
360:	learn: 0.0074380	total: 1m 1s	remai

498:	learn: 0.0040525	total: 1m 26s	remaining: 21.2s
499:	learn: 0.0040489	total: 1m 26s	remaining: 21.1s
500:	learn: 0.0040296	total: 1m 26s	remaining: 20.9s
501:	learn: 0.0040293	total: 1m 26s	remaining: 20.7s
502:	learn: 0.0040235	total: 1m 26s	remaining: 20.5s
503:	learn: 0.0040076	total: 1m 26s	remaining: 20.4s
504:	learn: 0.0040057	total: 1m 27s	remaining: 20.2s
505:	learn: 0.0040029	total: 1m 27s	remaining: 20s
506:	learn: 0.0039927	total: 1m 27s	remaining: 19.8s
507:	learn: 0.0039863	total: 1m 27s	remaining: 19.7s
508:	learn: 0.0039805	total: 1m 27s	remaining: 19.5s
509:	learn: 0.0039577	total: 1m 27s	remaining: 19.3s
510:	learn: 0.0039544	total: 1m 28s	remaining: 19.1s
511:	learn: 0.0039530	total: 1m 28s	remaining: 19s
512:	learn: 0.0039435	total: 1m 28s	remaining: 18.8s
513:	learn: 0.0039362	total: 1m 28s	remaining: 18.6s
514:	learn: 0.0039272	total: 1m 28s	remaining: 18.5s
515:	learn: 0.0039119	total: 1m 28s	remaining: 18.3s
516:	learn: 0.0038956	total: 1m 29s	remaining: 18.

In [ ]:
model1 = XGBClassifier(random_state=0, n_jobs=-1, learning_rate=0.01,
                  n_estimators=1000, max_depth=2, min_child_weight=0.5,
                  gamma=0, subsample=0.6, colsample_bytree=0.55,
                  reg_alpha=0.01, reg_lambda=100, objective= 'binary:logistic',
                  nthread=4, scale_pos_weight=1, seed=27)
model1 = model1.fit(S_train1, y_train)
y_pred1 = model1.predict_proba(S_test1)
y_pred1

In [41]:
models_0 = [
    GaussianNB(),
    
    LogisticRegression(random_state=27),
    
    CatBoostClassifier(random_state=27, depth=10, 
                       iterations=1000, l2_leaf_reg=10),

    ExtraTreesClassifier(random_state=27, n_jobs=-1, 
                         n_estimators=180, max_depth=5),
            
    RandomForestClassifier(random_state=27, n_jobs=-1, 
                           n_estimators=1000, max_depth=180),
        
    XGBClassifier(random_state=27, n_jobs=-1, learning_rate=0.01,
                  n_estimators=1000, max_depth=2, min_child_weight=0.5,
                  gamma=0, subsample=0.6, colsample_bytree=0.55,
                  reg_alpha=0.01, reg_lambda=100, objective= 'binary:logistic',
                  nthread=4, scale_pos_weight=1, seed=27),
    
    LGBMClassifier(random_state=27, n_jobs=-1, boosting_type="gbdt", learning_rate=0.01, n_estimators=420, 
                   num_leaves=4, is_unbalance=True, max_depth=8, feature_fraction=0.5, bagging_fraction=0.8, bagging_freq=15)
]

In [42]:
S_train0, S_test0 = stacking(models_0,                   
                           X_train[predictors], y_train, X_test[predictors],   # our data
                           regression=False, 
#                            Boolean indicating whether we want to use the function for regression. 
#                            In our case set to False since this is a classification
                           mode='oof_pred_bag', 
#                            using the earlier describe out-of-fold during cross-validation
                           needs_proba=True,
         
                           save_dir=None, 
#                            save the result to directory Boolean
                           metric=auc, 
#                            what evaluation metric to use (we imported the accuracy_score in the beginning)    
                           n_folds=5, 
#                            how many folds to use for cross-validation
                           stratified=True,
#                            whether to use stratified cross-validation            
                           shuffle=True, 
#                            whether to shuffle the data            
                           random_state=0,    
         
                           verbose=2 
#                           2 here refers to printing all info
                          )

task:         [classification]
n_classes:    [2]
metric:       [auc]
mode:         [oof_pred_bag]
n_models:     [7]

model  0:     [GaussianNB]
    fold  0:  [0.72637911]
    fold  1:  [0.72379062]
    fold  2:  [0.73593960]
    fold  3:  [0.71751770]
    fold  4:  [0.72627018]
    ----
    MEAN:     [0.72597945] + [0.00593028]
    FULL:     [0.72327601]

model  1:     [LogisticRegression]
    fold  0:  [0.84930829]
    fold  1:  [0.83411378]
    fold  2:  [0.81999949]
    fold  3:  [0.81512717]
    fold  4:  [0.83300204]
    ----
    MEAN:     [0.83031015] + [0.01199563]
    FULL:     [0.83014607]

model  2:     [CatBoostClassifier]
0:	learn: 0.6347114	total: 157ms	remaining: 2m 36s
1:	learn: 0.5829997	total: 308ms	remaining: 2m 33s
2:	learn: 0.5346019	total: 466ms	remaining: 2m 34s
3:	learn: 0.4911682	total: 621ms	remaining: 2m 34s
4:	learn: 0.4527080	total: 779ms	remaining: 2m 35s
5:	learn: 0.4183199	total: 880ms	remaining: 2m 25s
6:	learn: 0.3889147	total: 1.04s	remaining: 2m 27s
7

146:	learn: 0.0737011	total: 23.3s	remaining: 2m 15s
147:	learn: 0.0736599	total: 23.4s	remaining: 2m 14s
148:	learn: 0.0735595	total: 23.6s	remaining: 2m 14s
149:	learn: 0.0734238	total: 23.7s	remaining: 2m 14s
150:	learn: 0.0734208	total: 23.9s	remaining: 2m 14s
151:	learn: 0.0733077	total: 24.1s	remaining: 2m 14s
152:	learn: 0.0731376	total: 24.2s	remaining: 2m 14s
153:	learn: 0.0730070	total: 24.4s	remaining: 2m 13s
154:	learn: 0.0729727	total: 24.5s	remaining: 2m 13s
155:	learn: 0.0728686	total: 24.7s	remaining: 2m 13s
156:	learn: 0.0728459	total: 24.9s	remaining: 2m 13s
157:	learn: 0.0728206	total: 25s	remaining: 2m 13s
158:	learn: 0.0727822	total: 25.2s	remaining: 2m 13s
159:	learn: 0.0727129	total: 25.3s	remaining: 2m 13s
160:	learn: 0.0727072	total: 25.5s	remaining: 2m 12s
161:	learn: 0.0727018	total: 25.6s	remaining: 2m 12s
162:	learn: 0.0726361	total: 25.8s	remaining: 2m 12s
163:	learn: 0.0725996	total: 26s	remaining: 2m 12s
164:	learn: 0.0725276	total: 26.1s	remaining: 2m 1

304:	learn: 0.0671304	total: 48.3s	remaining: 1m 50s
305:	learn: 0.0670683	total: 48.5s	remaining: 1m 49s
306:	learn: 0.0670272	total: 48.7s	remaining: 1m 49s
307:	learn: 0.0670018	total: 48.8s	remaining: 1m 49s
308:	learn: 0.0669294	total: 49s	remaining: 1m 49s
309:	learn: 0.0668928	total: 49.1s	remaining: 1m 49s
310:	learn: 0.0668758	total: 49.3s	remaining: 1m 49s
311:	learn: 0.0668699	total: 49.4s	remaining: 1m 49s
312:	learn: 0.0668625	total: 49.6s	remaining: 1m 48s
313:	learn: 0.0668352	total: 49.8s	remaining: 1m 48s
314:	learn: 0.0668037	total: 49.9s	remaining: 1m 48s
315:	learn: 0.0667900	total: 50.1s	remaining: 1m 48s
316:	learn: 0.0667844	total: 50.2s	remaining: 1m 48s
317:	learn: 0.0667317	total: 50.4s	remaining: 1m 48s
318:	learn: 0.0667178	total: 50.5s	remaining: 1m 47s
319:	learn: 0.0666365	total: 50.7s	remaining: 1m 47s
320:	learn: 0.0665747	total: 50.9s	remaining: 1m 47s
321:	learn: 0.0665258	total: 51s	remaining: 1m 47s
322:	learn: 0.0665133	total: 51.2s	remaining: 1m 4

459:	learn: 0.0625812	total: 1m 14s	remaining: 1m 27s
460:	learn: 0.0625732	total: 1m 14s	remaining: 1m 27s
461:	learn: 0.0625314	total: 1m 14s	remaining: 1m 26s
462:	learn: 0.0624819	total: 1m 14s	remaining: 1m 26s
463:	learn: 0.0624781	total: 1m 14s	remaining: 1m 26s
464:	learn: 0.0624713	total: 1m 15s	remaining: 1m 26s
465:	learn: 0.0624615	total: 1m 15s	remaining: 1m 26s
466:	learn: 0.0624179	total: 1m 15s	remaining: 1m 26s
467:	learn: 0.0623790	total: 1m 15s	remaining: 1m 25s
468:	learn: 0.0623717	total: 1m 15s	remaining: 1m 25s
469:	learn: 0.0623286	total: 1m 15s	remaining: 1m 25s
470:	learn: 0.0623022	total: 1m 16s	remaining: 1m 25s
471:	learn: 0.0622987	total: 1m 16s	remaining: 1m 25s
472:	learn: 0.0622635	total: 1m 16s	remaining: 1m 25s
473:	learn: 0.0622614	total: 1m 16s	remaining: 1m 24s
474:	learn: 0.0622109	total: 1m 16s	remaining: 1m 24s
475:	learn: 0.0621929	total: 1m 16s	remaining: 1m 24s
476:	learn: 0.0621819	total: 1m 16s	remaining: 1m 24s
477:	learn: 0.0621364	total:

613:	learn: 0.0592203	total: 1m 38s	remaining: 1m 2s
614:	learn: 0.0592195	total: 1m 38s	remaining: 1m 1s
615:	learn: 0.0591614	total: 1m 39s	remaining: 1m 1s
616:	learn: 0.0591013	total: 1m 39s	remaining: 1m 1s
617:	learn: 0.0590547	total: 1m 39s	remaining: 1m 1s
618:	learn: 0.0590302	total: 1m 39s	remaining: 1m 1s
619:	learn: 0.0589564	total: 1m 39s	remaining: 1m 1s
620:	learn: 0.0589238	total: 1m 39s	remaining: 1m
621:	learn: 0.0588543	total: 1m 40s	remaining: 1m
622:	learn: 0.0588234	total: 1m 40s	remaining: 1m
623:	learn: 0.0588078	total: 1m 40s	remaining: 1m
624:	learn: 0.0587966	total: 1m 40s	remaining: 1m
625:	learn: 0.0587638	total: 1m 40s	remaining: 1m
626:	learn: 0.0587393	total: 1m 40s	remaining: 60s
627:	learn: 0.0587205	total: 1m 40s	remaining: 59.8s
628:	learn: 0.0586937	total: 1m 41s	remaining: 59.7s
629:	learn: 0.0586901	total: 1m 41s	remaining: 59.5s
630:	learn: 0.0586818	total: 1m 41s	remaining: 59.3s
631:	learn: 0.0586478	total: 1m 41s	remaining: 59.2s
632:	learn: 0

...

372:	learn: 0.0631430	total: 1m	remaining: 1m 42s
373:	learn: 0.0631086	total: 1m	remaining: 1m 41s
374:	learn: 0.0630639	total: 1m 1s	remaining: 1m 41s
375:	learn: 0.0630555	total: 1m 1s	remaining: 1m 41s
376:	learn: 0.0630033	total: 1m 1s	remaining: 1m 41s
377:	learn: 0.0630009	total: 1m 1s	remaining: 1m 41s
378:	learn: 0.0629908	total: 1m 1s	remaining: 1m 41s
379:	learn: 0.0629805	total: 1m 1s	remaining: 1m 40s
380:	learn: 0.0629778	total: 1m 2s	remaining: 1m 40s
381:	learn: 0.0629752	total: 1m 2s	remaining: 1m 40s
382:	learn: 0.0629587	total: 1m 2s	remaining: 1m 40s
383:	learn: 0.0629261	total: 1m 2s	remaining: 1m 40s
384:	learn: 0.0628775	total: 1m 2s	remaining: 1m 40s
385:	learn: 0.0628335	total: 1m 2s	remaining: 1m 39s
386:	learn: 0.0628020	total: 1m 3s	remaining: 1m 39s
387:	learn: 0.0627912	total: 1m 3s	remaining: 1m 39s
388:	learn: 0.0626745	total: 1m 3s	remaining: 1m 39s
389:	learn: 0.0626070	total: 1m 3s	remaining: 1m 39s
390:	learn: 0.0625733	total: 1m 3s	remaining: 1m 39s

525:	learn: 0.0591270	total: 1m 25s	remaining: 1m 17s
526:	learn: 0.0591236	total: 1m 25s	remaining: 1m 16s
527:	learn: 0.0591000	total: 1m 25s	remaining: 1m 16s
528:	learn: 0.0590837	total: 1m 26s	remaining: 1m 16s
529:	learn: 0.0590724	total: 1m 26s	remaining: 1m 16s
530:	learn: 0.0590168	total: 1m 26s	remaining: 1m 16s
531:	learn: 0.0589947	total: 1m 26s	remaining: 1m 16s
532:	learn: 0.0589695	total: 1m 26s	remaining: 1m 15s
533:	learn: 0.0589110	total: 1m 26s	remaining: 1m 15s
534:	learn: 0.0589101	total: 1m 27s	remaining: 1m 15s
535:	learn: 0.0588766	total: 1m 27s	remaining: 1m 15s
536:	learn: 0.0588715	total: 1m 27s	remaining: 1m 15s
537:	learn: 0.0588688	total: 1m 27s	remaining: 1m 15s
538:	learn: 0.0588314	total: 1m 27s	remaining: 1m 14s
539:	learn: 0.0587714	total: 1m 27s	remaining: 1m 14s
540:	learn: 0.0587493	total: 1m 27s	remaining: 1m 14s
541:	learn: 0.0587288	total: 1m 28s	remaining: 1m 14s
542:	learn: 0.0587211	total: 1m 28s	remaining: 1m 14s
543:	learn: 0.0587120	total:

680:	learn: 0.0549549	total: 1m 52s	remaining: 52.5s
681:	learn: 0.0548962	total: 1m 52s	remaining: 52.3s
682:	learn: 0.0548850	total: 1m 52s	remaining: 52.1s
683:	learn: 0.0548571	total: 1m 52s	remaining: 52s
684:	learn: 0.0548338	total: 1m 52s	remaining: 51.8s
685:	learn: 0.0548285	total: 1m 52s	remaining: 51.6s
686:	learn: 0.0548195	total: 1m 52s	remaining: 51.5s
687:	learn: 0.0548107	total: 1m 53s	remaining: 51.3s
688:	learn: 0.0548045	total: 1m 53s	remaining: 51.1s
689:	learn: 0.0547759	total: 1m 53s	remaining: 51s
690:	learn: 0.0547347	total: 1m 53s	remaining: 50.8s
691:	learn: 0.0547333	total: 1m 53s	remaining: 50.7s
692:	learn: 0.0547211	total: 1m 53s	remaining: 50.5s
693:	learn: 0.0547013	total: 1m 54s	remaining: 50.3s
694:	learn: 0.0547000	total: 1m 54s	remaining: 50.2s
695:	learn: 0.0546845	total: 1m 54s	remaining: 50s
696:	learn: 0.0546510	total: 1m 54s	remaining: 49.8s
697:	learn: 0.0545662	total: 1m 54s	remaining: 49.7s
698:	learn: 0.0545572	total: 1m 54s	remaining: 49.5s

838:	learn: 0.0511388	total: 2m 17s	remaining: 26.4s
839:	learn: 0.0511331	total: 2m 17s	remaining: 26.2s
840:	learn: 0.0511134	total: 2m 17s	remaining: 26.1s
841:	learn: 0.0510684	total: 2m 17s	remaining: 25.9s
842:	learn: 0.0510622	total: 2m 18s	remaining: 25.7s
843:	learn: 0.0510048	total: 2m 18s	remaining: 25.6s
844:	learn: 0.0509999	total: 2m 18s	remaining: 25.4s
845:	learn: 0.0509834	total: 2m 18s	remaining: 25.2s
846:	learn: 0.0509723	total: 2m 18s	remaining: 25.1s
847:	learn: 0.0509678	total: 2m 18s	remaining: 24.9s
848:	learn: 0.0509542	total: 2m 19s	remaining: 24.7s
849:	learn: 0.0509362	total: 2m 19s	remaining: 24.6s
850:	learn: 0.0508784	total: 2m 19s	remaining: 24.4s
851:	learn: 0.0508374	total: 2m 19s	remaining: 24.2s
852:	learn: 0.0508234	total: 2m 19s	remaining: 24.1s
853:	learn: 0.0507968	total: 2m 19s	remaining: 23.9s
854:	learn: 0.0507701	total: 2m 20s	remaining: 23.8s
855:	learn: 0.0507530	total: 2m 20s	remaining: 23.6s
856:	learn: 0.0507376	total: 2m 20s	remaining:

993:	learn: 0.0476079	total: 2m 42s	remaining: 982ms
994:	learn: 0.0475989	total: 2m 42s	remaining: 818ms
995:	learn: 0.0475805	total: 2m 43s	remaining: 655ms
996:	learn: 0.0475769	total: 2m 43s	remaining: 491ms
997:	learn: 0.0475474	total: 2m 43s	remaining: 327ms
998:	learn: 0.0475354	total: 2m 43s	remaining: 164ms
999:	learn: 0.0475148	total: 2m 43s	remaining: 0us
    fold  4:  [0.84545128]
    ----
    MEAN:     [0.83888742] + [0.00855405]
    FULL:     [0.83872255]

model  3:     [ExtraTreesClassifier]
    fold  0:  [0.82370963]
    fold  1:  [0.81896454]
    fold  2:  [0.80984381]
    fold  3:  [0.81178654]
    fold  4:  [0.81381173]
    ----
    MEAN:     [0.81562325] + [0.00505881]
    FULL:     [0.81530837]

model  4:     [RandomForestClassifier]
    fold  0:  [0.83694681]
    fold  1:  [0.82078335]
    fold  2:  [0.81699605]
    fold  3:  [0.81648036]
    fold  4:  [0.82274458]
    ----
    MEAN:     [0.82279023] + [0.00745420]
    FULL:     [0.82276108]

model  5:     [XGBCla

In [45]:
S_train0[1]

array([1.00000000e+00, 3.81938797e-22, 9.93696931e-01, 6.30306858e-03,
       9.95982777e-01, 4.01722295e-03, 9.86999588e-01, 1.30004121e-02,
       9.99000000e-01, 1.00000000e-03, 9.90699291e-01, 9.30068921e-03,
       8.19577029e-01, 1.80422971e-01])

In [57]:
modelLGBM = LGBMClassifier(
    boosting_type="gbdt",
    learning_rate=0.1,
    is_unbalance=True, 
    random_state=27, 
    n_estimators=50,
    num_leaves=30, 
    max_depth=8,
    feature_fraction=0.5,  
    bagging_fraction=0.8, 
    bagging_freq=15   
)

In [58]:
params_opt = {'n_estimators':range(100, 500, 80), 
              'num_leaves':range(5,30,5), 
              'learning_rate':[0.001,0.01,0.1]
             }
gridSearchCV = GridSearchCV(estimator = modelLGBM, param_grid = params_opt, scoring='roc_auc', 
                            n_jobs=-1, iid=False, verbose=1, cv=5)
gridSearchCV.fit(S_train0,y_train)
gridSearchCV.cv_results_, gridSearchCV.best_params_, gridSearchCV.best_score_

Fitting 5 folds for each of 75 candidates, totalling 375 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    9.8s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 375 out of 375 | elapsed:  4.0min finished


({'mean_fit_time': array([ 1.90353389,  3.13228722,  3.80660067,  4.32013507,  5.95402961,
          2.8463098 ,  4.60426855,  5.59266534,  6.73074732,  8.97086635,
          3.29540458,  6.19185133,  7.36238441,  9.19821033, 12.31794648,
          4.35373778,  7.89872875,  9.64624152, 11.84364405, 16.07093158,
          4.94170365,  8.96123729, 11.35962062, 13.92757769, 17.96364956,
          1.04595594,  2.41813345,  3.00997305,  3.64999146,  4.76006651,
          2.51957202,  4.58423805,  5.83266387,  6.96348495,  9.06164193,
          3.60580373,  6.34609852,  7.55334353,  9.53603077, 14.66432123,
          5.20723915,  9.93767843, 11.59058895, 12.55487981, 15.09249387,
          4.74610424,  9.42573328, 11.41249313, 14.53293991, 17.74908829,
          1.11745524,  2.48376799,  3.03153472,  3.52064886,  5.02098413,
          2.75624094,  4.164923  ,  5.09707012,  6.13643503,  8.40487237,
          3.21752639,  5.90186982,  7.00966115,  8.064574  , 11.02161026,
          3.64212174,

In [63]:
modelLGBM = LGBMClassifier(
    boosting_type="gbdt",
    learning_rate=0.01,
    is_unbalance=True, 
    random_state=27, 
    n_estimators=260,
    num_leaves=5, 
    max_depth=8,
    feature_fraction=0.5,  
    bagging_fraction=0.8, 
    bagging_freq=15   
)
params_opt = {'num_leaves':range(5,8)}
gridSearchCV = GridSearchCV(estimator = modelLGBM, param_grid = params_opt, scoring='roc_auc', 
                            n_jobs=-1, iid=False, verbose=1, cv=5)
gridSearchCV.fit(S_train0,y_train)
gridSearchCV.cv_results_, gridSearchCV.best_params_, gridSearchCV.best_score_

Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 out of  15 | elapsed:    3.0s remaining:    2.6s
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:    4.6s finished


({'mean_fit_time': array([2.43112054, 2.5983006 , 2.05808196]),
  'std_fit_time': array([0.02444111, 0.04175827, 0.50838257]),
  'mean_score_time': array([0.34370141, 0.35219426, 0.26341166]),
  'std_score_time': array([0.02155907, 0.01907865, 0.03761938]),
  'param_num_leaves': masked_array(data=[5, 6, 7],
               mask=[False, False, False],
         fill_value='?',
              dtype=object),
  'params': [{'num_leaves': 5}, {'num_leaves': 6}, {'num_leaves': 7}],
  'split0_test_score': array([0.76278591, 0.76331588, 0.7633905 ]),
  'split1_test_score': array([0.97383654, 0.97404745, 0.97382997]),
  'split2_test_score': array([0.83968348, 0.83893151, 0.8391808 ]),
  'split3_test_score': array([0.81838986, 0.81868718, 0.8184977 ]),
  'split4_test_score': array([0.82241135, 0.82192474, 0.82183597]),
  'mean_test_score': array([0.84342143, 0.84338135, 0.84334699]),
  'std_test_score': array([0.07013154, 0.07010577, 0.07002345]),
  'rank_test_score': array([1, 2, 3])},
 {'num_leave

In [ ]:
model = LGBMClassifier(
    boosting_type="gbdt",
    learning_rate=0.01,
    is_unbalance=True, 
    random_state=27, 
    n_estimators=420,
    num_leaves=30, 
    max_depth=8,
    feature_fraction=0.5,  
    bagging_fraction=0.8, 
    bagging_freq=15   
)
params_opt = {'num_leaves':range(3,8,1)}
gridSearchCV = GridSearchCV(estimator = model, param_grid = params_opt, scoring='roc_auc', 
                            n_jobs=-1, iid=False, verbose=1, cv=5)
gridSearchCV.fit(X_train_1,y_train)
gridSearchCV.cv_results_, gridSearchCV.best_params_, gridSearchCV.best_score_

In [ ]:
modelLGBM = LGBMClassifier(
    boosting_type="gbdt",
    learning_rate=0.01,
    is_unbalance=True, 
    random_state=27, 
    n_estimators=420,
    num_leaves=30, 
    max_depth=8,
    feature_fraction=0.5,  
    bagging_fraction=0.8, 
    bagging_freq=15   
)
modelLGBM = modelLGBM.fit(S_train0, y_train)
y_predLGBM = modelLGBM.predict_proba(S_test0)
y_predLGBM

In [ ]:
pd.DataFrame(y_pred0).to_csv("stacking_LGBM.csv")

In [ ]:
# params_opt = {'n_estimators':range(100, 500, 80), 
#               'num_leaves':range(5,30,5), 
#               'learning_rate':[0.001,0.01,0.1]
#              }
# gridSearchCV = GridSearchCV(estimator = modelLGBM, param_grid = params_opt, scoring='roc_auc', 
#                             n_jobs=-1, iid=False, verbose=1, cv=5)
# gridSearchCV.fit(S_train0,y_train)
# gridSearchCV.cv_results_, gridSearchCV.best_params_, gridSearchCV.best_score_

In [78]:
param_test0 = {'n_estimators':range(100, 150, 10)}
gsearch0 = GridSearchCV(estimator = XGBClassifier(learning_rate =0.1, n_estimators=90, max_depth=5,
                                                  min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
                                                  objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27),
                        param_grid = param_test0, scoring='roc_auc', n_jobs=4, iid=False, cv=5)
gsearch0.fit(S_train0,y_train)
gsearch0.cv_results_, gsearch0.best_params_, gsearch0.best_score_

({'mean_fit_time': array([ 7.93288631,  9.24515624, 10.20731792, 10.94927645, 10.91004896]),
  'std_fit_time': array([0.54215948, 0.35867759, 0.30275519, 0.27091288, 1.77300125]),
  'mean_score_time': array([0.04577909, 0.05106354, 0.05555296, 0.05775065, 0.05494723]),
  'std_score_time': array([0.00260841, 0.00395903, 0.00255576, 0.00415107, 0.01109963]),
  'param_n_estimators': masked_array(data=[100, 110, 120, 130, 140],
               mask=[False, False, False, False, False],
         fill_value='?',
              dtype=object),
  'params': [{'n_estimators': 100},
   {'n_estimators': 110},
   {'n_estimators': 120},
   {'n_estimators': 130},
   {'n_estimators': 140}],
  'split0_test_score': array([0.75941926, 0.75982679, 0.76015995, 0.76011977, 0.75867258]),
  'split1_test_score': array([0.96581224, 0.96596142, 0.96681103, 0.96627003, 0.96556199]),
  'split2_test_score': array([0.8355433 , 0.83474311, 0.83450351, 0.83569075, 0.83448986]),
  'split3_test_score': array([0.81497557, 0.

In [75]:
param_test1 = {
 'max_depth':range(1,4),
 'min_child_weight':range(0,4)
}
gsearch1 = GridSearchCV(estimator = XGBClassifier(learning_rate =0.1, n_estimators=100, max_depth=5,
                                                  min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
                                                  objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27),
                        param_grid = param_test1, scoring='roc_auc', n_jobs=4, iid=False, cv=5)
gsearch1.fit(S_train0,y_train)
gsearch1.cv_results_, gsearch1.best_params_, gsearch1.best_score_

({'mean_fit_time': array([2.68320656, 2.74143839, 2.8873909 , 2.79024148, 4.13389359,
         4.15754533, 4.18417778, 4.25718946, 5.75653605, 5.72680988,
         5.57501912, 5.31899261]),
  'std_fit_time': array([0.09374839, 0.10658117, 0.11693552, 0.13125074, 0.08361139,
         0.16035906, 0.14185693, 0.14669521, 0.50177429, 0.23603158,
         0.20976492, 0.38012848]),
  'mean_score_time': array([0.02037997, 0.02164369, 0.02104511, 0.02154241, 0.02663035,
         0.02772584, 0.02772579, 0.03002138, 0.03480811, 0.0340106 ,
         0.03321271, 0.03023696]),
  'std_score_time': array([0.00100997, 0.00171565, 0.00155846, 0.00232616, 0.00222242,
         0.00277822, 0.00255459, 0.00161804, 0.00260567, 0.0006623 ,
         0.00153126, 0.00634064]),
  'param_max_depth': masked_array(data=[1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3],
               mask=[False, False, False, False, False, False, False, False,
                     False, False, False, False],
         fill_value='?',
         

In [77]:
param_test2 = {
    'gamma':[i/10.0 for i in range(0,5)]
}
gsearch2 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=100, max_depth=1,
                                                  min_child_weight=0, gamma=0, subsample=0.8, colsample_bytree=0.8,
                                                  objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27),
                        param_grid = param_test2, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch2.fit(S_train0,y_train)
gsearch2.cv_results_, gsearch2.best_params_, gsearch2.best_score_

({'mean_fit_time': array([2.73106556, 2.89073982, 3.14223409, 2.92041607, 2.65621219]),
  'std_fit_time': array([0.05923212, 0.10243274, 0.07400078, 0.08378523, 0.57429309]),
  'mean_score_time': array([0.02157121, 0.02433772, 0.02583303, 0.02333775, 0.01956491]),
  'std_score_time': array([0.00134733, 0.00259019, 0.00275384, 0.00223944, 0.00328013]),
  'param_gamma': masked_array(data=[0.0, 0.1, 0.2, 0.3, 0.4],
               mask=[False, False, False, False, False],
         fill_value='?',
              dtype=object),
  'params': [{'gamma': 0.0},
   {'gamma': 0.1},
   {'gamma': 0.2},
   {'gamma': 0.3},
   {'gamma': 0.4}],
  'split0_test_score': array([0.76165019, 0.76165019, 0.76165019, 0.76165019, 0.76165019]),
  'split1_test_score': array([0.973253, 0.973253, 0.973253, 0.973253, 0.973253]),
  'split2_test_score': array([0.83991084, 0.83991084, 0.83991084, 0.83991084, 0.83991084]),
  'split3_test_score': array([0.8182162, 0.8182162, 0.8182162, 0.8182162, 0.8182162]),
  'split4_test

In [79]:
param_test3 = {
    'gamma':[i/100.0 for i in range(0,5)]
}
gsearch3 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=100, max_depth=1,
                                                  min_child_weight=0, gamma=0, subsample=0.8, colsample_bytree=0.8,
                                                  objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27),
                        param_grid = param_test3, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch3.fit(X_train[predictors],y_train)
gsearch3.cv_results_, gsearch3.best_params_, gsearch3.best_score_

({'mean_fit_time': array([28.64884944, 29.7524044 , 30.3108726 , 30.21212621, 26.74674683]),
  'std_fit_time': array([1.5527259 , 0.90797177, 1.36293904, 0.97686759, 5.97624097]),
  'mean_score_time': array([0.33991361, 0.33712049, 0.37113194, 0.35497332, 0.27861996]),
  'std_score_time': array([0.02708619, 0.01647812, 0.03473362, 0.01803004, 0.06716688]),
  'param_gamma': masked_array(data=[0.0, 0.01, 0.02, 0.03, 0.04],
               mask=[False, False, False, False, False],
         fill_value='?',
              dtype=object),
  'params': [{'gamma': 0.0},
   {'gamma': 0.01},
   {'gamma': 0.02},
   {'gamma': 0.03},
   {'gamma': 0.04}],
  'split0_test_score': array([0.73445198, 0.73445198, 0.73445198, 0.73445198, 0.73445198]),
  'split1_test_score': array([0.95010886, 0.95010886, 0.95010886, 0.95010886, 0.95010886]),
  'split2_test_score': array([0.84055198, 0.84055198, 0.84055198, 0.84055198, 0.84055198]),
  'split3_test_score': array([0.80288074, 0.80288074, 0.80288074, 0.80288074, 

In [82]:
param_test4 = {'n_estimators':range(120, 960, 120)}
gsearch4 = GridSearchCV(estimator = XGBClassifier(learning_rate =0.1, n_estimators=90, max_depth=1,
                                                  min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
                                                  objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27),
                        param_grid = param_test4, scoring='roc_auc', n_jobs=4, iid=False, cv=5)
gsearch4.fit(S_train0,y_train)
gsearch4.cv_results_, gsearch4.best_params_, gsearch4.best_score_

({'mean_fit_time': array([ 3.37682385,  6.87826366, 10.23454223, 13.9390923 , 17.26403198,
         21.22291641, 22.45526009]),
  'std_fit_time': array([0.1399323 , 0.36736408, 0.27254452, 0.31106343, 0.10271863,
         0.19934159, 2.61702761]),
  'mean_score_time': array([0.02777157, 0.03341222, 0.04028993, 0.04857149, 0.0601409 ,
         0.07081399, 0.07311215]),
  'std_score_time': array([0.00335908, 0.00484477, 0.00248934, 0.00281101, 0.00502326,
         0.00363525, 0.01337494]),
  'param_n_estimators': masked_array(data=[120, 240, 360, 480, 600, 720, 840],
               mask=[False, False, False, False, False, False, False],
         fill_value='?',
              dtype=object),
  'params': [{'n_estimators': 120},
   {'n_estimators': 240},
   {'n_estimators': 360},
   {'n_estimators': 480},
   {'n_estimators': 600},
   {'n_estimators': 720},
   {'n_estimators': 840}],
  'split0_test_score': array([0.76190306, 0.76102251, 0.76094758, 0.76048622, 0.76020944,
         0.76025281,

In [83]:
param_test4 = {
    'subsample':[i/10.0 for i in range(6,10)],
    'colsample_bytree':[i/10.0 for i in range(6,10)]
}
gsearch4 = GridSearchCV(estimator = XGBClassifier(learning_rate=0.1, n_estimators=120, max_depth=1,
                                                  min_child_weight=0, gamma=0, subsample=0.8, colsample_bytree=0.8,
                                                  objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27),
                        param_grid = param_test4, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch4.fit(S_train0,y_train)
gsearch4.cv_results_, gsearch4.best_params_, gsearch4.best_score_

({'mean_fit_time': array([3.2424871 , 3.05747714, 3.30259147, 3.18431129, 3.35605321,
         3.41669946, 3.51354804, 3.39087076, 3.5382823 , 3.5338933 ,
         3.60919061, 3.75939631, 4.03367705, 3.70573959, 3.88237567,
         3.72592049]),
  'std_fit_time': array([0.07551351, 0.08129552, 0.25448302, 0.17417517, 0.16451366,
         0.12752006, 0.11494657, 0.07983422, 0.05452349, 0.09244941,
         0.13253033, 0.08514439, 0.141823  , 0.17281167, 0.03855872,
         0.24833883]),
  'mean_score_time': array([0.02478657, 0.02373667, 0.02313786, 0.02303963, 0.02454429,
         0.0263299 , 0.02593083, 0.02293868, 0.02433815, 0.02553167,
         0.0249362 , 0.02483506, 0.02742825, 0.02393599, 0.02543335,
         0.02124567]),
  'std_score_time': array([0.00264367, 0.00330156, 0.00171614, 0.0016808 , 0.00287597,
         0.00184973, 0.00199499, 0.00141071, 0.00139336, 0.00079806,
         0.0015748 , 0.00142149, 0.00109385, 0.00063068, 0.00223315,
         0.00436494]),
  'param_c

In [85]:
param_test5 = {
    'subsample':[0.55,0.6,0.65],
    'colsample_bytree':[0.85,0.9,0.95]
}
gsearch5 = GridSearchCV(estimator = XGBClassifier(learning_rate=0.1, n_estimators=120, max_depth=1,
                                                  min_child_weight=0, gamma=0, subsample=0.8, colsample_bytree=0.8,
                                                  objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27),
                        param_grid = param_test5, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch5.fit(S_train0,y_train)
gsearch5.cv_results_, gsearch5.best_params_, gsearch5.best_score_

({'mean_fit_time': array([3.22393942, 3.27163486, 3.56550493, 3.70025191, 3.57986727,
         3.58275809, 3.62385001, 3.65567064, 3.42791739]),
  'std_fit_time': array([0.12635462, 0.13295844, 0.15518855, 0.17896136, 0.059331  ,
         0.07637645, 0.07328365, 0.1327487 , 0.64800708]),
  'mean_score_time': array([0.02357664, 0.02273927, 0.02553525, 0.02503467, 0.02413526,
         0.02483525, 0.02712741, 0.02473383, 0.02194214]),
  'std_score_time': array([0.00183477, 0.00203458, 0.00308123, 0.00210021, 0.00116321,
         0.00337432, 0.00230899, 0.00182811, 0.00553427]),
  'param_colsample_bytree': masked_array(data=[0.85, 0.85, 0.85, 0.9, 0.9, 0.9, 0.95, 0.95, 0.95],
               mask=[False, False, False, False, False, False, False, False,
                     False],
         fill_value='?',
              dtype=object),
  'param_subsample': masked_array(data=[0.55, 0.6, 0.65, 0.55, 0.6, 0.65, 0.55, 0.6, 0.65],
               mask=[False, False, False, False, False, False, Fals

In [86]:
param_test6 = {
    'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100],
    'reg_lambda':[1e-5, 1e-2, 0.1, 1, 100]
}
gsearch6 = GridSearchCV(estimator = XGBClassifier(learning_rate=0.1, n_estimators=120, max_depth=1,
                                                  min_child_weight=0.5, gamma=0, subsample=0.6, colsample_bytree=0.95,
                                                  objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27),
                        param_grid = param_test6, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch6.fit(S_train0,y_train)
gsearch6.cv_results_, gsearch6.best_params_, gsearch6.best_score_

({'mean_fit_time': array([4.28468518, 3.62443838, 3.55063372, 3.49817405, 3.54415064,
         3.66164098, 3.64259477, 3.53577189, 3.68109097, 3.69614658,
         3.71839318, 3.63441386, 3.60768223, 3.72318473, 3.61885843,
         3.65206795, 3.88046618, 3.69954228, 3.77055554, 3.70502768,
         3.73046126, 3.70104346, 3.68019261, 3.70892262, 3.72614026]),
  'std_fit_time': array([0.50146918, 0.28448442, 0.15407734, 0.1612703 , 0.11518024,
         0.24550388, 0.04271704, 0.14071772, 0.09803068, 0.11899789,
         0.13414866, 0.05157887, 0.04389794, 0.06677207, 0.06939675,
         0.07259717, 0.09942934, 0.0485335 , 0.14604493, 0.06782039,
         0.10560992, 0.0904087 , 0.02913749, 0.09163376, 0.3805009 ]),
  'mean_score_time': array([0.02174206, 0.02373629, 0.02523408, 0.02433491, 0.02323928,
         0.02453451, 0.0236383 , 0.02443604, 0.02314124, 0.02423663,
         0.0267283 , 0.02513266, 0.02343845, 0.02343936, 0.02663069,
         0.02563295, 0.0261301 , 0.02553463, 0.

In [89]:
param_test7 = {
    'reg_alpha':[0.5, 1, 2],
    'reg_lambda':[0.5, 1, 2]
}
gsearch7 = GridSearchCV(estimator = XGBClassifier(learning_rate=0.1, n_estimators=120, max_depth=1,
                                                  min_child_weight=0, gamma=0, subsample=0.6, colsample_bytree=0.95,
                                                  objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27),
                        param_grid = param_test7, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch7.fit(S_train0,y_train)
gsearch7.cv_results_, gsearch7.best_params_, gsearch7.best_score_

({'mean_fit_time': array([3.49959698, 3.49564672, 3.64205503, 3.64205475, 3.86645451,
         3.74358282, 3.8010293 , 3.86086884, 3.48248215]),
  'std_fit_time': array([0.15848839, 0.14667906, 0.11331619, 0.14955251, 0.1020624 ,
         0.05846626, 0.0698684 , 0.29451503, 0.65657917]),
  'mean_score_time': array([0.02333751, 0.02453423, 0.02333755, 0.02493296, 0.02573094,
         0.02553172, 0.02433524, 0.02513299, 0.02034574]),
  'std_score_time': array([0.00256997, 0.00286296, 0.00119688, 0.00178382, 0.00317872,
         0.00256989, 0.00149258, 0.00159587, 0.00371042]),
  'param_reg_alpha': masked_array(data=[0.5, 0.5, 0.5, 1, 1, 1, 2, 2, 2],
               mask=[False, False, False, False, False, False, False, False,
                     False],
         fill_value='?',
              dtype=object),
  'param_reg_lambda': masked_array(data=[0.5, 1, 2, 0.5, 1, 2, 0.5, 1, 2],
               mask=[False, False, False, False, False, False, False, False,
                     False],
   

In [91]:
param_test8 = {'n_estimators':range(90, 120, 10)}
gsearch8 = GridSearchCV(estimator = XGBClassifier(learning_rate =0.1, n_estimators=120, max_depth=1, reg_alpha=0.5,
                                                  reg_lambda=1, min_child_weight=0, gamma=0, subsample=0.6, 
                                                  colsample_bytree=0.95, objective= 'binary:logistic', nthread=4, 
                                                  scale_pos_weight=1, seed=27),
                        param_grid = param_test8, scoring='roc_auc', n_jobs=4, iid=False, cv=5)
gsearch8.fit(S_train0,y_train)
gsearch8.cv_results_, gsearch8.best_params_, gsearch8.best_score_

({'mean_fit_time': array([2.55356598, 2.92377644, 2.94531636]),
  'std_fit_time': array([0.04574025, 0.08386971, 0.25213534]),
  'mean_score_time': array([0.02014613, 0.02373652, 0.01974711]),
  'std_score_time': array([0.00171568, 0.00324096, 0.00425033]),
  'param_n_estimators': masked_array(data=[90, 100, 110],
               mask=[False, False, False],
         fill_value='?',
              dtype=object),
  'params': [{'n_estimators': 90},
   {'n_estimators': 100},
   {'n_estimators': 110}],
  'split0_test_score': array([0.7633602 , 0.76330555, 0.76327564]),
  'split1_test_score': array([0.97398418, 0.97359152, 0.97329145]),
  'split2_test_score': array([0.84243291, 0.84255159, 0.84251907]),
  'split3_test_score': array([0.81953227, 0.8195919 , 0.81913533]),
  'split4_test_score': array([0.82252564, 0.82271894, 0.82270325]),
  'mean_test_score': array([0.84436704, 0.8443519 , 0.84418495]),
  'std_test_score': array([0.06994321, 0.06979351, 0.06972306]),
  'rank_test_score': array([

In [93]:
param_test9 = {
    'learning_rate':[0.05,0.1,0.5]
}
gsearch9 = GridSearchCV(estimator = XGBClassifier(learning_rate=0.1, n_estimators=90, max_depth=1, reg_alpha=0.5,
                                                  reg_lambda=1, min_child_weight=0, gamma=0, subsample=0.6, 
                                                  colsample_bytree=0.95, objective= 'binary:logistic', nthread=4, 
                                                  scale_pos_weight=1,seed=27),
                        param_grid = param_test9, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch9.fit(S_train0,y_train)
gsearch9.cv_results_, gsearch9.best_params_, gsearch9.best_score_

({'mean_fit_time': array([2.59724326, 2.66985579, 2.49951181]),
  'std_fit_time': array([0.12417623, 0.08341411, 0.22393293]),
  'mean_score_time': array([0.01994662, 0.02214084, 0.02094426]),
  'std_score_time': array([0.00063098, 0.00193388, 0.00356837]),
  'param_learning_rate': masked_array(data=[0.05, 0.1, 0.5],
               mask=[False, False, False],
         fill_value='?',
              dtype=object),
  'params': [{'learning_rate': 0.05},
   {'learning_rate': 0.1},
   {'learning_rate': 0.5}],
  'split0_test_score': array([0.76098769, 0.7633602 , 0.75579726]),
  'split1_test_score': array([0.95985721, 0.97398418, 0.96982978]),
  'split2_test_score': array([0.83648304, 0.84243291, 0.82952474]),
  'split3_test_score': array([0.81692302, 0.81953227, 0.81696486]),
  'split4_test_score': array([0.81872468, 0.82252564, 0.81606103]),
  'mean_test_score': array([0.83859513, 0.84436704, 0.83763554]),
  'std_test_score': array([0.0657212 , 0.06994321, 0.07089618]),
  'rank_test_score':

In [46]:
model0 = XGBClassifier(random_state=0, n_jobs=-1, learning_rate=0.01,
                  n_estimators=1000, max_depth=2, min_child_weight=0.5,
                  gamma=0, subsample=0.6, colsample_bytree=0.55,
                  reg_alpha=0.01, reg_lambda=100, objective= 'binary:logistic',
                  nthread=4, scale_pos_weight=1, seed=27)
model0 = model0.fit(S_train0, y_train)
y_pred0 = model0.predict_proba(S_test0)
y_pred0

array([[0.9433285 , 0.05667151],
       [0.99807656, 0.00192343],
       [0.99447024, 0.00552976],
       ...,
       [0.76565605, 0.23434393],
       [0.83117235, 0.16882767],
       [0.8700824 , 0.12991764]], dtype=float32)

In [47]:
pd.DataFrame(y_pred0).to_csv("stacking_try_0.csv")

In [98]:
model0 = XGBClassifier(random_state=0, n_jobs=-1, learning_rate=0.1,
                  n_estimators=90, max_depth=1, min_child_weight=0,
                  gamma=0, subsample=0.6, colsample_bytree=0.95,
                  reg_alpha=0.05, reg_lambda=1, objective= 'binary:logistic',
                  nthread=4, scale_pos_weight=1, seed=27)
model0 = model0.fit(S_train0, y_train)
y_pred0 = model0.predict_proba(S_test0)
y_pred0

array([[0.93352026, 0.06647972],
       [0.99844444, 0.00155555],
       [0.9927708 , 0.00722922],
       ...,
       [0.7808881 , 0.2191119 ],
       [0.85873854, 0.14126147],
       [0.9279684 , 0.07203162]], dtype=float32)

In [99]:
pd.DataFrame(y_pred0).to_csv("stacking_try_xgb.csv")